## **Imports**

In [1]:
import pandas as pd
import numpy as np
import importlib
from application.modeling import disbursements
from application.modeling import interest_income
from application.modeling import other_income
from application.modeling import helper
from application.modeling import depreciation
from application.modeling import expenses
from application.modeling import income_statement
from application.modeling import direct_cashflow
from application.modeling import balance_sheet
from application.modeling import statement_of_cashflows
from application.modeling import loan_book
from application.modeling import borrowings

pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:,.2f}".format

## **Inputs**

In [2]:
VALUATION_DATE = "2023-01"
MONTHS_TO_FORECAST = 12
IMTT = 0.01

In [3]:
parameters = pd.read_csv("./data_templates/parameters.csv", index_col=0)
parameters.columns = pd.period_range(VALUATION_DATE,periods=int(parameters.columns[-1]), freq="M")
parameters.head()

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
NEW_INVENTORY,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
INVENTORY_USED,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_INTERGROUP_RECEIVABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_OTHER_PAYABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_OTHER_RECEIVABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [4]:
details_of_existing_assets = pd.read_csv("./data_templates/details_of_existing_assets.csv")
details_of_existing_assets.head()

,Asset ID,Acquisition Date,Life,Book Value,Net Value,Salvage Value,Method,Depreciation
0,FF-HRE-0024,01/09/2015,10,380.00,75.96,0,straight_line,NaN
1,FF-HRE-0025,01/09/2015,10,370.00,74.04,0,straight_line,NaN
2,FF-HRE-0026,01/09/2015,10,370.00,74.04,0,straight_line,NaN
3,FF-HRE-0027,01/09/2015,10,630.00,126.00,0,straight_line,NaN
4,FF-HRE-0028,01/09/2015,10,630.00,126.00,0,straight_line,NaN


In [5]:
details_of_new_assets = pd.read_csv("./data_templates/details_of_new_assets.csv")
details_of_new_assets.head()

,Asset ID,Acquisition Date,Life,Book Value,Net Value,Salvage Value,Method,Depreciation


In [6]:
details_of_existing_long_term_borrowing = pd.read_csv("./data_templates/details_of_existing_long_term_borrowing.csv")
details_of_existing_long_term_borrowing.head()

,Effective Date,Institution,Nominal Amount,Interest Rate,Tenure,Repayment Frequency,Frequency,Method
0,31/12/2022,ZIMNAT LIFE ASSURANCE,376455,0.06,60,Monthly (P+I),12,reducing_balance
1,06/05/2022,MASAWARA,5613,0.06,60,Monthly (P+I),12,reducing_balance
2,31/12/2022,Grandre,160000,0.06,36,Quaterly (I+Bullet P),4,straight_line
3,31/12/2022,MINERVA,15000,0.08,36,Bullet (P+I),0,straight_line
4,10/09/2022,ZAM-CARGO,364000,0.05,36,Annually(I) Principal rollover,12,straight_line


In [7]:
details_of_new_long_term_borrowing = pd.read_csv("./data_templates/details_of_new_long_term_borrowing.csv")
details_of_new_long_term_borrowing.head()

,Effective Date,Institution,Nominal Amount,Interest Rate,Tenure,Repayment Frequency,Frequency,Method
0,31/12/2023,ZIMNAT LIFE ASSURANCE,376455,0.06,60,Monthly (P+I),12,reducing_balance
1,06/05/2024,MASAWARA,5613,0.06,60,Monthly (P+I),12,reducing_balance


In [8]:
details_of_new_short_term_borrowing = pd.read_csv("./data_templates/details_of_new_short_term_borrowing.csv")
details_of_new_short_term_borrowing.head()

,Effective Date,Institution,Nominal Amount,Interest Rate,Tenure,Repayment Frequency,Frequency,Method
0,31/12/2023,ZIMNAT LIFE ASSURANCE,376455,0.06,12,Monthly (P+I),12,reducing_balance
1,06/05/2024,MASAWARA,5613,0.06,12,Monthly (P+I),12,reducing_balance


In [9]:
details_of_existing_short_term_borrowing = pd.read_csv("./data_templates/details_of_existing_short_term_borrowing.csv")
details_of_existing_short_term_borrowing.head()

,Effective Date,Institution,Nominal Amount,Interest Rate,Tenure,Repayment Frequency,Frequency,Method
0,24/04/2023,FCB,"120,383.84",0.18,3,Bullet (P+I),0,straight_line
1,02/03/2023,ZIMNAT LIFE ASSURANCE,"50,000.00",0.14,6,Monthly (P+I),12,reducing_balance
2,31/12/2022,Zimnat Life Assurance,"20,000.00",0.08,12,Annually(I) Principal rollover,1,straight_line
3,31/12/2022,ZLA-Cargo,"124,575.00",0.10,12,Annually(I) Principal rollover,1,straight_line
4,31/12/2022,ZIMNAT LION INSURANCE,"50,000.00",0.08,12,Bi-annual(I)Principal at maturity,2,straight_line


In [10]:
static_inputs_income_statement = pd.read_csv("./data_templates/static_inputs_income_statement.csv", index_col=0)
static_inputs_income_statement.head()

,Jan-23,Feb-23,Mar-23,Apr-23,May-23,Jun-23,Jul-23,Aug-23,Sep-23,Oct-23,Nov-23,Dec-23
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
CILL,278.00,"1,313.00",277.00,"1,910.00","3,048.00","6,169.00",528.00,222.00,913.00,882.00,876.00,862.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"
Retrenchments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Staff Welfare,"1,081.56","3,861.43","3,645.71","3,203.79","4,191.68","2,658.42","19,268.45","16,790.41","16,910.35","17,031.13","17,152.78","17,275.30"


In [11]:
variable_inputs_income_statement = pd.read_csv("./data_templates/variable_inputs_income_statement.csv", index_col=0)
variable_inputs_income_statement.head()

,1,2,3,4,5,6
Travel Costs,0.00,0.00,0.00,0.00,0.00,0.00
Entertainment,0.00,721.39,67.00,0.00,"4,530.00",124.00
Marketing Costs,"3,104.93","4,869.97","8,450.63","4,098.90","5,692.00",195.60
Group Marketing Costs,"7,145.92","7,543.70","12,847.68","9,277.20","6,097.62","15,314.38"
Rental Costs,910.55,"1,168.40",583.42,"1,479.51","1,601.02","1,315.55"


In [12]:
opening_balances = pd.read_csv("./data_templates/opening_balances.csv")
opening_balances

,TRADE_RECEIVABLES,TRADE_PAYABLES,LOAN_BOOK,CASH_ON_HAND,ISSUED_SHARE_CAPITAL,SHARE_PREMIUM,OTHER_COMPONENTS_OF_EQUITY,TREASURY_SHARES,RETAINED_EARNINGS,LONG_TERM_LOANS,SHORT_TERM_LOANS,INTERCOMPANY_LOANS,DEFERED_TAXATION,OTHER_RECEIVABLES,OTHER_PAYABLES,PROVISION_FOR_TAXATION,PROVISION_FOR_CREDIT_LOSS,INVESTMENT_IN_SUBSIDIARIES,INVESTMENT_IN_ASSOCIATES,INVESTMENT_PROPERTIES,EQUITY_INVESTMENTS,LONG_TERM_MONEY_MARKET_INVESTMENTS,LOANS_TO_RELATED_ENTITIES,TREASURY_SHARES.1,INTANGIBLE_ASSETS,INVENTORIES,OTHER_RECEIVABLES.1,INTERGROUP_RECEIVABLES,SHORT_TERM_MONEY_MARKET_INVESTMENTS
0,1092458,938595,2806038,331856,7050,110,59784,0,876511,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
existing_loans = pd.read_csv("./data_templates/existing_loans.csv")
existing_loans.head()

,Loan Number,Disbursement Date,Loan Term,Loan Amount,Closing Balance,Interest Rate,Admin Fee,Credit Insurance Fee,Frequency
0,100120644000001,13/10/2021,60,"20,000.00","-14,769.50",0.03,0.07,0.02,12
1,100120644000002,15/10/2021,60,"12,000.00","-8,855.52",0.03,0.07,0.02,12
2,100120644000003,25/10/2021,60,"20,000.00","-14,708.39",0.03,0.07,0.02,12
3,100120644000004,26/10/2021,60,"20,000.00","-14,703.34",0.03,0.07,0.02,12
4,100120644000005,28/10/2021,60,"20,000.00","-14,693.08",0.03,0.07,0.02,12


## **New Disbursements**

In [14]:
new_disbursements_df  = disbursements.calculate_new_disbursements(parameters=parameters)
new_disbursements_df.head()

,b2b_disbursements,sme_disbursements,consumer_ssb_disbursements,consumer_pvt_disbursements,total
2023-01,"57,011.05","360,000.00","48,000.00","60,000.00","537,011.05"
2023-02,"27,480.25","432,000.00","60,000.00","75,000.00","609,480.25"
2023-03,"27,480.25","480,000.00","72,000.00","108,000.00","723,480.25"
2023-04,"34,900.00","360,000.00","80,000.00","120,000.00","634,900.00"
2023-05,"66,173.00","324,000.00","96,000.00","144,000.00","678,173.00"


### **Calculate Repayment Amount For New Disbursements**

In [15]:
monthly_repayment_new_disbursements_df = interest_income.calculate_monthly_repayments_new_disbursements(new_disbursements_df=new_disbursements_df, parameters=parameters)
monthly_repayment_new_disbursements_df.head()

,sme_monthly_repayment,b2b_monthly_repayment,consumer_ssb_monthly_repayment,consumer_pvt_monthly_repayment,total
2023-01,"81,451.18","6,683.43","8,006.34","10,007.93","106,148.89"
2023-02,"97,741.42","3,221.52","10,007.93","12,509.91","123,480.78"
2023-03,"108,601.58","3,221.52","12,009.51","18,014.27","141,846.88"
2023-04,"81,451.18","4,091.34","11,172.05","16,758.08","113,472.66"
2023-05,"73,306.06","7,757.49","13,406.46","20,109.69","114,579.72"


### **Generate Loan Schedules For New Disbursements**

In [16]:
loan_schedules_for_all_new_disbursements = (
    interest_income.generate_loan_schedules_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        parameters=parameters,
        monthly_repayment_new_disbursements_df=monthly_repayment_new_disbursements_df,
        months_to_forecast=MONTHS_TO_FORECAST,
    )
)
loan_schedules_for_all_new_disbursements.keys()

dict_keys(['sme_loan_schedules', 'b2b_loan_schedules', 'consumer_ssb_loan_schedules', 'consumer_pvt_loan_schedules'])

In [17]:
loan_schedules_for_all_new_disbursements['sme_loan_schedules'].keys()

dict_keys(['interest', 'capital_repayments', 'outstanding_at_start'])

### **Capital Repayment New Disbursement**

In [18]:
capital_repayment_new_disbursements_df = interest_income.generate_capital_repayment_new_disbursements_df(loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements)
capital_repayment_new_disbursements_df.head()

,sme_capital_repayments,b2b_capital_repayments,consumer_pvt_capital_repayments,consumer_ssb_capital_repayments,total
Feb-2023,"47,251.18","4,973.10","4,607.93","3,686.34","60,518.56"
Mar-2023,"108,441.46","7,519.41","10,782.55","8,626.04","135,369.47"
Apr-2023,"181,744.98","10,142.11","20,047.25","14,931.90","226,866.24"
May-2023,"246,261.93","13,490.72","27,809.58","20,247.82","307,810.06"
Jun-2023,"312,182.88","19,667.74","37,462.14","26,836.59","396,149.35"


### **Interest Income New Disbursements**


In [19]:
interest_income_new_disbursements_df = interest_income.generate_interest_income_new_disbursements_df(
    loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements
)

interest_income_new_disbursements_df

,sme_interest_income,b2b_interest_income,consumer_pvt_interest_income,consumer_ssb_interest_income,total
Feb-2023,"34,200.00","1,710.33","5,400.00","4,320.00","45,630.33"
Mar-2023,"70,751.14","2,385.55","11,735.29","9,388.23","94,260.20"
Apr-2023,"106,049.20","2,984.37","20,484.86","15,091.89","144,610.31"
May-2023,"122,983.43","3,727.11","29,480.60","20,948.01","177,139.15"
Jun-2023,"130,368.54","5,307.58","39,937.74","27,765.71","203,379.57"
Jul-2023,"136,621.17","5,278.82","51,146.15","33,990.42","227,036.56"
Aug-2023,"139,333.04","5,841.63","63,757.21","41,008.77","249,940.64"
Sep-2023,"157,637.89","5,319.26","76,342.80","47,971.09","287,271.04"
Oct-2023,"184,153.18","6,440.29","89,540.97","54,220.18","334,354.61"
Nov-2023,"236,008.72","7,516.35","101,099.04","59,691.86","404,315.97"


### **Calculate Admin Fee On New Disbursements**

In [20]:
admin_fee_for_all_new_disbursements_df = other_income.calculate_admin_fee_for_all_new_disbursements(
    new_disbursements_df=new_disbursements_df,
    parameters=parameters,
    months_to_forecast=MONTHS_TO_FORECAST,
)

admin_fee_for_all_new_disbursements_df

,sme_admin_fee,b2b_admin_fee,consumer_ssb_admin_fee,consumer_pvt_admin_fee,total
Jan-2023,"4,098.00",389.39,252.18,372.55,"5,112.12"
Feb-2023,"9,015.60",577.08,567.42,838.23,"10,998.32"
Mar-2023,"14,479.60",764.77,945.69,"1,508.81","17,698.87"
Apr-2023,"18,577.60","1,003.13","1,366.00","2,253.90","23,200.63"
May-2023,"22,265.80","1,455.09","1,870.37","3,148.01","28,739.27"
Jun-2023,"26,568.70","1,582.88","2,374.74","4,153.88","34,680.20"
Jul-2023,"27,251.70","1,860.52","2,963.17","5,327.40","37,402.79"
Aug-2023,"29,107.18","1,919.80","3,593.63","6,584.74","41,205.36"
Sep-2023,"31,611.52","2,363.75","4,224.09","7,981.79","46,181.15"
Oct-2023,"38,783.02","2,841.85","4,854.55","9,378.83","55,858.25"


### **Calculate Credit Life Insurance On New Disbursement**

In [21]:
credit_insurance_fee_for_all_new_disbursements = other_income.calculate_credit_insurance_fee_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        parameters=parameters,
        months_to_forecast=MONTHS_TO_FORECAST,
    )

credit_insurance_fee_for_all_new_disbursements

,sme_credit_insurance_fee,b2b_credit_insurance_fee,consumer_ssb_credit_insurance_fee,consumer_pvt_credit_insurance_fee,total
Jan-2023,"4,800.00",456.09,295.38,436.36,"5,987.84"
Feb-2023,"10,560.00",675.93,664.62,981.82,"12,882.36"
Mar-2023,"16,960.00",895.77,"1,107.69","1,767.27","20,730.74"
Apr-2023,"21,760.00","1,174.97","1,600.00","2,640.00","27,174.97"
May-2023,"26,080.00","1,704.36","2,190.77","3,687.27","33,662.40"
Jun-2023,"31,120.00","1,854.03","2,781.54","4,865.45","40,621.02"
Jul-2023,"31,920.00","2,179.23","3,470.77","6,240.00","43,810.00"
Aug-2023,"34,093.33","2,248.67","4,209.23","7,712.73","48,263.96"
Sep-2023,"37,026.67","2,768.67","4,947.69","9,349.09","54,092.12"
Oct-2023,"45,426.67","3,328.67","5,686.15","10,985.45","65,426.95"


### **Calculate For Credit Loss For New Disbursements**

In [22]:
provision_for_credit_loss_for_all_new_disbursements = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df, parameters=parameters
    )
)
provision_for_credit_loss_for_all_new_disbursements

,sme_provision_for_credit_loss,b2b_provision_for_credit_loss,consumer_ssb_provision_for_credit_loss,consumer_pvt_provision_for_credit_loss,total
Jan-2023,"7,200.00",0.00,960.00,"1,800.00","9,960.00"
Feb-2023,"8,640.00",0.00,"1,200.00","2,250.00","12,090.00"
Mar-2023,"9,600.00",0.00,"1,440.00","3,240.00","14,280.00"
Apr-2023,"7,200.00",0.00,"1,600.00","3,600.00","12,400.00"
May-2023,"6,480.00",0.00,"1,920.00","4,320.00","12,720.00"
Jun-2023,"7,560.00",0.00,"1,920.00","4,860.00","14,340.00"
Jul-2023,"8,400.00",0.00,"2,240.00","5,670.00","16,310.00"
Aug-2023,"11,900.00",0.00,"2,400.00","6,075.00","20,375.00"
Sep-2023,"14,000.00",0.00,"2,400.00","6,750.00","23,150.00"
Oct-2023,"19,800.00",0.00,"2,400.00","6,750.00","28,950.00"


## **Existing Loans**

### **Generate Loan Schedules For Existing Loans**


In [23]:
existing_loans

,Loan Number,Disbursement Date,Loan Term,Loan Amount,Closing Balance,Interest Rate,Admin Fee,Credit Insurance Fee,Frequency
0,100120644000001,13/10/2021,60,"20,000.00","-14,769.50",0.03,0.07,0.02,12
1,100120644000002,15/10/2021,60,"12,000.00","-8,855.52",0.03,0.07,0.02,12
2,100120644000003,25/10/2021,60,"20,000.00","-14,708.39",0.03,0.07,0.02,12
3,100120644000004,26/10/2021,60,"20,000.00","-14,703.34",0.03,0.07,0.02,12
4,100120644000005,28/10/2021,60,"20,000.00","-14,693.08",0.03,0.07,0.02,12
...,...,...,...,...,...,...,...,...,...
4451,100120708001044,28/09/2023,9,456.82,-456.82,0.09,0.07,0.02,12
4452,100120708001045,26/09/2023,9,348.52,-348.52,0.09,0.07,0.02,12
4453,100120708001046,30/09/2023,9,289.73,-289.73,0.09,0.07,0.02,12
4454,100120708001047,30/09/2023,9,568.21,-568.21,0.09,0.07,0.02,12


In [24]:
existing_loans = helper.columns_to_snake_case(existing_loans)

In [25]:
existing_loans_schedules = borrowings.calculate_reducing_balance_loans_schedules(
    interest_rates=existing_loans["interest_rate"],
    effective_dates=existing_loans["disbursement_date"],
    frequencies=existing_loans["frequency"],
    loan_identifiers=existing_loans["loan_number"],
    tenures=existing_loans["loan_term"],
    amounts=existing_loans["loan_amount"],
)
existing_loans_schedules.keys()

dict_keys(['outstanding_balance_at_start', 'capital_repayments', 'interest_payments', 'repayments'])

### **Capital Repayment for Existing Loans**

In [26]:
capital_repayment_existing_loans = existing_loans_schedules['capital_repayments'].sum()
capital_repayment_existing_loans

Jan-2019      629.39
Feb-2019    3,245.44
Mar-2019    8,021.09
Apr-2019    9,315.64
May-2019   12,737.70
              ...   
Jun-2027    4,173.62
Jul-2027    2,747.85
Aug-2027    1,318.57
Sep-2027    1,321.82
Oct-2027    1,325.08
Length: 106, dtype: float64

### **Intererest Income For Existing Loans**

In [27]:
interest_income_existing_loans = existing_loans_schedules['interest_payments'].sum()
interest_income_existing_loans

Jan-2019    15.93
Feb-2019   102.93
Mar-2019   283.64
Apr-2019   242.25
May-2019   296.48
            ...  
Jun-2027    26.85
Jul-2027    16.56
Aug-2027     9.78
Sep-2027     6.53
Oct-2027     3.27
Length: 106, dtype: float64

### **Calculate Other Income Existing Loans**

In [28]:
other_income_existing_loans_df = other_income.calculate_other_income_existing_loans(
    existing_loans=existing_loans,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
other_income_existing_loans_df

,credit_insurance_fee_existing_loans,admin_fee_existing_loans,total
Jan-2023,"13,261.21","53,278.86","66,540.07"
Feb-2023,"12,595.01","50,602.30","63,197.31"
Mar-2023,"11,859.65","47,647.91","59,507.56"
Apr-2023,"11,152.20","44,805.60","55,957.79"
May-2023,"9,413.75","37,821.11","47,234.85"
Jun-2023,"8,789.06","35,311.33","44,100.39"
Jul-2023,"7,915.09","31,800.03","39,715.12"
Aug-2023,"6,949.18","27,919.35","34,868.52"
Sep-2023,"5,881.44","23,629.53","29,510.97"
Oct-2023,"5,094.43","20,467.61","25,562.03"


## **Interest Income**

In [29]:
total_interest_income = interest_income.aggregate_new_and_existing_loans_interest_income(
    interest_income_new_disbursements_df=interest_income_new_disbursements_df,
    interest_income_existing_loans= interest_income_existing_loans,
    valuation_date= VALUATION_DATE, 
    months_to_forecast= MONTHS_TO_FORECAST
)
total_interest_income

Jan-2023     9,587.46
Feb-2023    55,784.14
Mar-2023   105,246.42
Apr-2023   155,191.25
May-2023   187,266.47
Jun-2023   215,650.68
Jul-2023   238,969.54
Aug-2023   261,806.00
Sep-2023   298,832.36
Oct-2023   343,657.74
Nov-2023   411,687.24
Dec-2023   481,262.20
dtype: float64

## **Other Income**

In [30]:
other_income_df = other_income.aggregate_other_income(
    admin_fee_for_all_new_disbursements_df=admin_fee_for_all_new_disbursements_df,
    admin_fee_existing_loans=other_income_existing_loans_df['admin_fee_existing_loans'],
    credit_insurance_fee_existing_loans=other_income_existing_loans_df['credit_insurance_fee_existing_loans'],
    credit_insurance_fee_for_all_new_disbursements_df=credit_insurance_fee_for_all_new_disbursements, 
    valuation_date=VALUATION_DATE, 
    months_to_forecast=MONTHS_TO_FORECAST
)
other_income_df

,admin_fee,credit_insurance_fee,total
Jan-2023,"63,503.09","25,236.88","88,739.98"
Feb-2023,"72,598.93","38,359.74","110,958.67"
Mar-2023,"83,045.64","53,321.13","136,366.77"
Apr-2023,"91,206.86","65,502.14","156,709.00"
May-2023,"95,299.65","76,738.54","172,038.19"
Jun-2023,"104,671.73","90,031.10","194,702.83"
Jul-2023,"106,605.60","95,535.09","202,140.69"
Aug-2023,"110,330.06","103,477.10","213,807.16"
Sep-2023,"115,991.83","114,065.68","230,057.51"
Oct-2023,"132,184.12","135,948.32","268,132.43"


## **Depreciation**

In [31]:
details_of_existing_assets = helper.columns_to_snake_case(details_of_existing_assets)
details_of_new_assets = helper.columns_to_snake_case(details_of_new_assets)

In [32]:
depreciations_and_nbvs = depreciation.calculate_depreciations_and_nbvs(
    details_of_existing_assets=details_of_existing_assets,
    details_of_new_assets=details_of_new_assets,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST
)
depreciations_and_nbvs.keys()

dict_keys(['nbvs', 'dpns'])

In [33]:
depreciations_and_nbvs['nbvs']

,CE-HRE-0039,CE-HRE-0040,CE-HRE-0041,CE-HRE-0042,CE-HRE-0043,CE-HRE-0044,CE-HRE-0045,CE-HRE-0046,CE-HRE-0047,CE-HRE-0048,CE-HRE-0049,CE-HRE-0050,CE-HRE-0051,CE-HRE-0052,CE-HRE-0053,CE-HRE-0070,CE-HRE-0071,CE-HRE-0075,CE-HRE-0077,CE-HRE-0081,CE-HRE-0085,CE-HRE-0086,CE-HRE-0087,CE-HRE-0089,CE-HRE-0090,CE-HRE-0091,CE-HRE-0092,CE-HRE-0093,CE-HRE-0094,CE-HRE-0095,CE-HRE-0096,CE-HRE-0097,CE-HRE-0098,CE-HRE-0100,CE-HRE-0101,CE-HRE-0102,CE-HRE-0103,CE-HRE-0104,CE-HRE-0105,CE-HRE-0106,CE-HRE-0107,CE-HRE-0108,CE-HRE-0109,CE-HRE-0110,CE-HRE-0111,CE-HRE-0112,CE-ZFX-0003,CE-ZFX-0004,CE-ZFX-0006,CE-ZFX-0007,CS-HRE-0003,CS-HRE-0004,CS-HRE-0005,CS-ZFX-0002,EL-BYO-0036,EL-HRE-0007,EL-HRE-0017,EL-HRE-0032,EL-HRE-0037,EL-HRE-0038,FF-BYO-0057,FF-BYO-0058,FF-BYO-0059,FF-BYO-0060,FF-BYO-0061,FF-BYO-0062,FF-BYO-0063,FF-BYO-0064,FF-BYO-0065,FF-BYO-0066,FF-BYO-0067,FF-BYO-0068,FF-BYO-0069,FF-BYO-0070,FF-BYO-0075,FF-BYO-0146,FF-HRE-0001,FF-HRE-0002,FF-HRE-0003,FF-HRE-0004,FF-HRE-0005,FF-HRE-0006,FF-HRE-0007,FF-HRE-0008,FF-HRE-0009,FF-HRE-0010,FF-HRE-0011,FF-HRE-0012,FF-HRE-0013,FF-HRE-0014,FF-HRE-0015,FF-HRE-0016,FF-HRE-0017,FF-HRE-0018,FF-HRE-0019,FF-HRE-0020,FF-HRE-0021,FF-HRE-0022,FF-HRE-0023,FF-HRE-0024,FF-HRE-0025,FF-HRE-0026,FF-HRE-0027,FF-HRE-0028,FF-HRE-0029,FF-HRE-0030,FF-HRE-0031,FF-HRE-0032,FF-HRE-0033,FF-HRE-0034,FF-HRE-0035,FF-HRE-0036,FF-HRE-0037,FF-HRE-0041,FF-HRE-0042,FF-HRE-0043,FF-HRE-0045,FF-HRE-0050,FF-HRE-0051,FF-HRE-0052,FF-HRE-0054,FF-HRE-0055,FF-HRE-0071,FF-HRE-0072,FF-HRE-0073,FF-HRE-0074,FF-HRE-0076,FF-HRE-0086,FF-HRE-0087,FF-HRE-0088,FF-HRE-0089,FF-HRE-0090,FF-HRE-0091,FF-HRE-0092,FF-HRE-0093,FF-HRE-0094,FF-HRE-0095,FF-HRE-0096,FF-HRE-0097,FF-HRE-0098,FF-HRE-0099,FF-HRE-0100,FF-HRE-0101,FF-HRE-0102,FF-HRE-0103,FF-HRE-0104,FF-HRE-0105,FF-HRE-0107,FF-HRE-0108,FF-HRE-0109,FF-HRE-0110,FF-HRE-0111,FF-HRE-0112,FF-HRE-0113,FF-HRE-0114,FF-HRE-0115,FF-HRE-0116,FF-HRE-0117,FF-HRE-0118,FF-HRE-0119,FF-HRE-0120,FF-HRE-0121,FF-HRE-0122,FF-HRE-0123,FF-HRE-0124,FF-HRE-0125,FF-HRE-0126,FF-HRE-0127,FF-HRE-0128,FF-HRE-0129,FF-HRE-0130,FF-HRE-0131,FF-HRE-0132,FF-HRE-0133,FF-HRE-0137,FF-HRE-0138,FF-HRE-0140,FF-HRE-0141,FF-HRE-0143,FF-HRE-0144,FF-HRE-0145,FF-HRE-0147,FF-HRE-0148,FF-HRE-0149,FF-HRE-0150,FF-HRE-0151,FF-HRE-0152,FF-HRE-0153,FF-HRE-0154,FF-HRE-0155,FF-HRE-0156,FF-HRE-0157,FF-HRE-0158,FF-HRE-0159,FF-HRE-0160,FF-HRE-0161,FF-HRE-0162,FF-HRE-0163,FF-HRE-0164,FF-HRE-0165,FF-HRE-0166,FF-HRE-0167,FF-HRE-0168,FF-HRE-0169,FF-HRE-0170,FF-HRE-0171,FF-HRE-0172,FF-HRE-0173,FF-HRE-0174,FF-HRE-0175,FF-HRE-0176,FF-HRE-0177,FF-HRE-0178,FF-HRE-0179,FF-HRE-0180,FF-HRE-0181,FF-HRE-0182,FF-HRE-0183,FF-HRE-0184,FF-HRE-0186,FF-HRE-0187,FF-HRE-0188,FF-HRE-0189,FF-HRE-0190,FF-HRE-0191,FF-HRE-0192,FF-HRE-0193,FF-HRE-0194,FF-HRE-0195,FF-HRE-0196,FF-HRE-0198,FF-HRE-0199,FF-HRE-0201,FF-HRE-0202,FF-HRE-0203,FF-HRE-0204,FF-ZFX-0001,FF-ZFX-0002,FF-ZFX-0003,FF-ZFX-0004,FF-ZFX-0007,MV-HRE-0003,MV-HRE-0006,MV-HRE-0008,MV-HRE-0009,total
Jan-2023,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,44.09,"1,401.87","1,606.82","3,861.00","10,191.43","6,200.10","32,025.00","11,215.16","139,727.90","101,695.40","107,055.17","148,353.15","148,353.15","148,353.15","148,353.15","148,353.14","148,353.14","148,353.14","1,679,763.48","3,529,883.10","891,919.58","891,919.58","891,919.58","891,919.58","891,919.58","891,919.58","891,919.58","891,919.58","1,337,675.23","1,016,693.88","10,129,620.41","2,177.06","2,177.06",635.46,"2,518.75","1,026.55",840.00,"2,287.07","11,586.96","12,303,570.33",71.15,168.34,"1,419.49","99,600.17","12,303,570.33",21.86,22.56,21.86,97.81,46.03,46.03,46.03,46.03,46.03,46.03,46.03,46.03,46.03,47.90,607.75,640.50,72.16,72.16,72.16,54.17,54.17,54.17,54.17,54.17,54.17,54.17,54.17,34.16,34.16,34.16,34.16,34.16,34.16,30.31,30.31,30.32,30.31,46.34,46.34,72.79,70.96,70.96,120.75,120.75,120.75,120.75,21.04,"1,082.39",529.82,392.04,462.00,567.95,96.00,390.00,66.00,266.04,183.95,"1,993.42",127.78,127.78,40.10,40.

In [34]:
depreciations_and_nbvs['dpns']

,CE-HRE-0039,CE-HRE-0040,CE-HRE-0041,CE-HRE-0042,CE-HRE-0043,CE-HRE-0044,CE-HRE-0045,CE-HRE-0046,CE-HRE-0047,CE-HRE-0048,CE-HRE-0049,CE-HRE-0050,CE-HRE-0051,CE-HRE-0052,CE-HRE-0053,CE-HRE-0070,CE-HRE-0071,CE-HRE-0075,CE-HRE-0077,CE-HRE-0081,CE-HRE-0085,CE-HRE-0086,CE-HRE-0087,CE-HRE-0089,CE-HRE-0090,CE-HRE-0091,CE-HRE-0092,CE-HRE-0093,CE-HRE-0094,CE-HRE-0095,CE-HRE-0096,CE-HRE-0097,CE-HRE-0098,CE-HRE-0100,CE-HRE-0101,CE-HRE-0102,CE-HRE-0103,CE-HRE-0104,CE-HRE-0105,CE-HRE-0106,CE-HRE-0107,CE-HRE-0108,CE-HRE-0109,CE-HRE-0110,CE-HRE-0111,CE-HRE-0112,CE-ZFX-0003,CE-ZFX-0004,CE-ZFX-0006,CE-ZFX-0007,CS-HRE-0003,CS-HRE-0004,CS-HRE-0005,CS-ZFX-0002,EL-BYO-0036,EL-HRE-0007,EL-HRE-0017,EL-HRE-0032,EL-HRE-0037,EL-HRE-0038,FF-BYO-0057,FF-BYO-0058,FF-BYO-0059,FF-BYO-0060,FF-BYO-0061,FF-BYO-0062,FF-BYO-0063,FF-BYO-0064,FF-BYO-0065,FF-BYO-0066,FF-BYO-0067,FF-BYO-0068,FF-BYO-0069,FF-BYO-0070,FF-BYO-0075,FF-BYO-0146,FF-HRE-0001,FF-HRE-0002,FF-HRE-0003,FF-HRE-0004,FF-HRE-0005,FF-HRE-0006,FF-HRE-0007,FF-HRE-0008,FF-HRE-0009,FF-HRE-0010,FF-HRE-0011,FF-HRE-0012,FF-HRE-0013,FF-HRE-0014,FF-HRE-0015,FF-HRE-0016,FF-HRE-0017,FF-HRE-0018,FF-HRE-0019,FF-HRE-0020,FF-HRE-0021,FF-HRE-0022,FF-HRE-0023,FF-HRE-0024,FF-HRE-0025,FF-HRE-0026,FF-HRE-0027,FF-HRE-0028,FF-HRE-0029,FF-HRE-0030,FF-HRE-0031,FF-HRE-0032,FF-HRE-0033,FF-HRE-0034,FF-HRE-0035,FF-HRE-0036,FF-HRE-0037,FF-HRE-0041,FF-HRE-0042,FF-HRE-0043,FF-HRE-0045,FF-HRE-0050,FF-HRE-0051,FF-HRE-0052,FF-HRE-0054,FF-HRE-0055,FF-HRE-0071,FF-HRE-0072,FF-HRE-0073,FF-HRE-0074,FF-HRE-0076,FF-HRE-0086,FF-HRE-0087,FF-HRE-0088,FF-HRE-0089,FF-HRE-0090,FF-HRE-0091,FF-HRE-0092,FF-HRE-0093,FF-HRE-0094,FF-HRE-0095,FF-HRE-0096,FF-HRE-0097,FF-HRE-0098,FF-HRE-0099,FF-HRE-0100,FF-HRE-0101,FF-HRE-0102,FF-HRE-0103,FF-HRE-0104,FF-HRE-0105,FF-HRE-0107,FF-HRE-0108,FF-HRE-0109,FF-HRE-0110,FF-HRE-0111,FF-HRE-0112,FF-HRE-0113,FF-HRE-0114,FF-HRE-0115,FF-HRE-0116,FF-HRE-0117,FF-HRE-0118,FF-HRE-0119,FF-HRE-0120,FF-HRE-0121,FF-HRE-0122,FF-HRE-0123,FF-HRE-0124,FF-HRE-0125,FF-HRE-0126,FF-HRE-0127,FF-HRE-0128,FF-HRE-0129,FF-HRE-0130,FF-HRE-0131,FF-HRE-0132,FF-HRE-0133,FF-HRE-0137,FF-HRE-0138,FF-HRE-0140,FF-HRE-0141,FF-HRE-0143,FF-HRE-0144,FF-HRE-0145,FF-HRE-0147,FF-HRE-0148,FF-HRE-0149,FF-HRE-0150,FF-HRE-0151,FF-HRE-0152,FF-HRE-0153,FF-HRE-0154,FF-HRE-0155,FF-HRE-0156,FF-HRE-0157,FF-HRE-0158,FF-HRE-0159,FF-HRE-0160,FF-HRE-0161,FF-HRE-0162,FF-HRE-0163,FF-HRE-0164,FF-HRE-0165,FF-HRE-0166,FF-HRE-0167,FF-HRE-0168,FF-HRE-0169,FF-HRE-0170,FF-HRE-0171,FF-HRE-0172,FF-HRE-0173,FF-HRE-0174,FF-HRE-0175,FF-HRE-0176,FF-HRE-0177,FF-HRE-0178,FF-HRE-0179,FF-HRE-0180,FF-HRE-0181,FF-HRE-0182,FF-HRE-0183,FF-HRE-0184,FF-HRE-0186,FF-HRE-0187,FF-HRE-0188,FF-HRE-0189,FF-HRE-0190,FF-HRE-0191,FF-HRE-0192,FF-HRE-0193,FF-HRE-0194,FF-HRE-0195,FF-HRE-0196,FF-HRE-0198,FF-HRE-0199,FF-HRE-0201,FF-HRE-0202,FF-HRE-0203,FF-HRE-0204,FF-ZFX-0001,FF-ZFX-0002,FF-ZFX-0003,FF-ZFX-0004,FF-ZFX-0007,MV-HRE-0003,MV-HRE-0006,MV-HRE-0008,MV-HRE-0009,total
Jan-2023,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,22.04,477.91,267.81,429.00,636.97,364.71,"1,601.25",659.71,"3,493.20","2,542.39","2,611.10","3,532.22","3,532.22","3,532.22","3,532.22","3,532.22","3,532.22","3,532.22","32,303.12","63,033.63","15,927.14","15,927.14","15,927.14","15,927.14","15,927.14","15,927.14","15,927.14","15,927.14","26,228.93","19,551.81","177,712.01",197.92,197.92,52.95,193.75,342.19,60.00,762.35,827.64,"361,869.23",2.63,3.92,19.99,"3,434.49","361,869.23",0.70,0.70,0.70,3.15,1.49,1.49,1.49,1.49,1.49,1.49,1.49,1.49,1.49,1.54,19.61,10.50,3.14,3.14,3.14,2.35,2.35,2.35,2.35,2.35,2.35,2.35,2.35,1.48,1.48,1.48,1.48,1.48,1.48,1.32,1.32,1.32,1.32,2.02,2.02,3.17,3.08,3.08,5.25,5.25,5.25,5.25,0.92,45.98,22.08,16.33,19.25,23.67,4.00,16.25,2.75,11.08,7.67,77.34,4.92,4.92,1.48,1.48,2.49,2.04,1.00,1.00,13.04,8.79,22.83,1.91,3.81,3.66,2.53,104.55,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,3.66,6.17,5.62,4.48,1.75,4.83,1.50,13.00,5.96,8.67,30.33,5.20,5.96,8.67,34.67,27.08,13.

## **Salaries**

In [35]:
salaries_and_pension_and_statutory_contributions_df  = expenses.calculate_salaries_and_pension_and_statutory_contributions(
    new_disbursements_df=new_disbursements_df,
    parameters=parameters,
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)
salaries_and_pension_and_statutory_contributions_df

,agent_commission,credit_officer_salaries,credit_officer_commission,other_staff_salaries,total,pensions_and_statutory_contributions
Jan-2023,"5,400.00",600.00,"3,600.00","17,000.00","26,600.00","3,021.76"
Feb-2023,"6,750.00",600.00,"4,320.00","17,000.00","28,670.00","3,256.91"
Mar-2023,"9,000.00",600.00,"4,800.00","17,000.00","31,400.00","3,567.04"
Apr-2023,"10,000.00",600.00,"3,600.00","17,000.00","31,200.00","3,544.32"
May-2023,"12,000.00",900.00,"3,240.00","17,000.00","33,140.00","3,764.70"
Jun-2023,"12,900.00",900.00,"3,780.00","17,000.00","34,580.00","3,928.29"
Jul-2023,"15,050.00","1,200.00","4,200.00","17,000.00","37,450.00","4,254.32"
Aug-2023,"16,125.00","1,400.00","5,950.00","17,000.00","40,475.00","4,597.96"
Sep-2023,"17,250.00","1,400.00","7,000.00","17,000.00","42,650.00","4,845.04"
Oct-2023,"17,250.00","1,800.00","9,900.00","17,000.00","45,950.00","5,219.92"


## **Provisions**


In [36]:
provision_for_credit_loss_for_all_new_disbursements = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df, parameters=parameters
    )
)
provision_for_credit_loss_for_all_new_disbursements

,sme_provision_for_credit_loss,b2b_provision_for_credit_loss,consumer_ssb_provision_for_credit_loss,consumer_pvt_provision_for_credit_loss,total
Jan-2023,"7,200.00",0.00,960.00,"1,800.00","9,960.00"
Feb-2023,"8,640.00",0.00,"1,200.00","2,250.00","12,090.00"
Mar-2023,"9,600.00",0.00,"1,440.00","3,240.00","14,280.00"
Apr-2023,"7,200.00",0.00,"1,600.00","3,600.00","12,400.00"
May-2023,"6,480.00",0.00,"1,920.00","4,320.00","12,720.00"
Jun-2023,"7,560.00",0.00,"1,920.00","4,860.00","14,340.00"
Jul-2023,"8,400.00",0.00,"2,240.00","5,670.00","16,310.00"
Aug-2023,"11,900.00",0.00,"2,400.00","6,075.00","20,375.00"
Sep-2023,"14,000.00",0.00,"2,400.00","6,750.00","23,150.00"
Oct-2023,"19,800.00",0.00,"2,400.00","6,750.00","28,950.00"


## **Calculating Finance Costs**

In [37]:
details_of_existing_long_term_borrowing = helper.columns_to_snake_case(
    details_of_existing_long_term_borrowing
)
details_of_existing_short_term_borrowing = helper.columns_to_snake_case(
    details_of_existing_short_term_borrowing
)
details_of_new_short_term_borrowing = helper.columns_to_snake_case(
    details_of_new_short_term_borrowing
)
details_of_new_long_term_borrowing = helper.columns_to_snake_case(
    details_of_new_long_term_borrowing
)

In [38]:
details_of_long_term_borrowings = pd.concat(
    [details_of_existing_long_term_borrowing, details_of_new_long_term_borrowing]
).reset_index(drop=True)
details_of_long_term_borrowings

,effective_date,institution,nominal_amount,interest_rate,tenure,repayment_frequency,frequency,method
0,31/12/2022,ZIMNAT LIFE ASSURANCE,376455,0.06,60,Monthly (P+I),12,reducing_balance
1,06/05/2022,MASAWARA,5613,0.06,60,Monthly (P+I),12,reducing_balance
2,31/12/2022,Grandre,160000,0.06,36,Quaterly (I+Bullet P),4,straight_line
3,31/12/2022,MINERVA,15000,0.08,36,Bullet (P+I),0,straight_line
4,10/09/2022,ZAM-CARGO,364000,0.05,36,Annually(I) Principal rollover,12,straight_line
5,05/04/2023,ZIMNAT LIFE ASSURANCE,30000,0.14,24,Monthly (P+I),4,reducing_balance
6,31/12/2023,ZIMNAT LIFE ASSURANCE,376455,0.06,60,Monthly (P+I),12,reducing_balance
7,06/05/2024,MASAWARA,5613,0.06,60,Monthly (P+I),12,reducing_balance


In [39]:
details_of_existing_short_term_borrowing

,effective_date,institution,nominal_amount,interest_rate,tenure,repayment_frequency,frequency,method
0,24/04/2023,FCB,"120,383.84",0.18,3,Bullet (P+I),0,straight_line
1,02/03/2023,ZIMNAT LIFE ASSURANCE,"50,000.00",0.14,6,Monthly (P+I),12,reducing_balance
2,31/12/2022,Zimnat Life Assurance,"20,000.00",0.08,12,Annually(I) Principal rollover,1,straight_line
3,31/12/2022,ZLA-Cargo,"124,575.00",0.10,12,Annually(I) Principal rollover,1,straight_line
4,31/12/2022,ZIMNAT LION INSURANCE,"50,000.00",0.08,12,Bi-annual(I)Principal at maturity,2,straight_line
5,31/12/2022,ZIMNAT LION INSURANCE,"92,800.00",0.06,12,Bi-annual(I)Principal at maturity,2,straight_line
6,31/12/2022,Grandre,"100,000.00",0.12,12,Quaterly (I+Bullet P),4,straight_line
7,23/04/2023,FCB,"18,429.93",1.50,12,Bullet (P+I),0,straight_line
8,17/12/2022,NURTURE,"500,000.00",0.20,12,Quaterly (I+Bullet P),4,straight_line
9,26/01/2023,NMB,"450,000.00",0.18,12,Quaterly (I+Bullet P),4,straight_line


In [40]:
details_of_new_short_term_borrowing

,effective_date,institution,nominal_amount,interest_rate,tenure,repayment_frequency,frequency,method
0,31/12/2023,ZIMNAT LIFE ASSURANCE,376455,0.06,12,Monthly (P+I),12,reducing_balance
1,06/05/2024,MASAWARA,5613,0.06,12,Monthly (P+I),12,reducing_balance


In [41]:
details_of_short_term_borrowings = pd.concat(
    [details_of_existing_short_term_borrowing, details_of_new_short_term_borrowing]
).reset_index(drop=True)
details_of_short_term_borrowings

,effective_date,institution,nominal_amount,interest_rate,tenure,repayment_frequency,frequency,method
0,24/04/2023,FCB,"120,383.84",0.18,3,Bullet (P+I),0,straight_line
1,02/03/2023,ZIMNAT LIFE ASSURANCE,"50,000.00",0.14,6,Monthly (P+I),12,reducing_balance
2,31/12/2022,Zimnat Life Assurance,"20,000.00",0.08,12,Annually(I) Principal rollover,1,straight_line
3,31/12/2022,ZLA-Cargo,"124,575.00",0.10,12,Annually(I) Principal rollover,1,straight_line
4,31/12/2022,ZIMNAT LION INSURANCE,"50,000.00",0.08,12,Bi-annual(I)Principal at maturity,2,straight_line
5,31/12/2022,ZIMNAT LION INSURANCE,"92,800.00",0.06,12,Bi-annual(I)Principal at maturity,2,straight_line
6,31/12/2022,Grandre,"100,000.00",0.12,12,Quaterly (I+Bullet P),4,straight_line
7,23/04/2023,FCB,"18,429.93",1.50,12,Bullet (P+I),0,straight_line
8,17/12/2022,NURTURE,"500,000.00",0.20,12,Quaterly (I+Bullet P),4,straight_line
9,26/01/2023,NMB,"450,000.00",0.18,12,Quaterly (I+Bullet P),4,straight_line


In [42]:
long_term_borrowings_schedules = borrowings.calculate_borrowings_schedules(details_of_long_term_borrowings)
long_term_borrowings_schedules.keys()

dict_keys(['interest_payments', 'capital_repayments'])

In [43]:
short_term_borrowings_schedules = borrowings.calculate_borrowings_schedules(details_of_short_term_borrowings)
short_term_borrowings_schedules.keys()

dict_keys(['interest_payments', 'capital_repayments'])

In [44]:
capital_repayment_borrowings_df = pd.concat(
    [
        long_term_borrowings_schedules["capital_repayments"],
        short_term_borrowings_schedules["capital_repayments"],
    ],
).fillna(0)

capital_repayment_borrowings_df.loc['total'] = capital_repayment_borrowings_df.sum()

In [45]:
finance_costs_df = pd.concat(
    [
        long_term_borrowings_schedules["interest_payments"],
        short_term_borrowings_schedules["interest_payments"],
    ],
).fillna(0)


finance_costs_df.loc["total"] = finance_costs_df.sum()
finance_costs_df

,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025,Jul-2025,Aug-2025,Sep-2025,Oct-2025,Nov-2025,Dec-2025,Jan-2026,Feb-2026,Mar-2026,Apr-2026,May-2026,Jun-2026,Jul-2026,Aug-2026,Sep-2026,Oct-2026,Nov-2026,Dec-2026,Jan-2027,Feb-2027,Mar-2027,Apr-2027,May-2027,Jun-2027,Jul-2027,Aug-2027,Sep-2027,Oct-2027,Nov-2027,Dec-2027,Jan-2028,Feb-2028,Mar-2028,Apr-2028,May-2028,Jun-2028,Jul-2028,Aug-2028,Sep-2028,Oct-2028,Nov-2028,Dec-2028,Jan-2029,Feb-2029,Mar-2029,Apr-2029
Grandre,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
MINERVA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZAM-CARGO,0.00,0.00,0.00,0.00,"1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,832.41","1,806.04","1,779.54","1,752.91","1,726.16","1,699.27","1,672.25","1,645.10","1,617.82","1,590.40","1,562.85","1,535.17","1,507.35","1,479.40","1,451.31","1,423.08","1,394.72","1,366.22","1,337.58","1,308.80","1,279.88","1,250.82","1,221.61","1,192.27","1,162.78","1,133.15","1,103.38","1,073.46","1,043.39","1,013.18",982.82,952.32,921.66,890.86,859.90,828.80,797.54,766.13,734.57,702.86,670.99,638.96,606.78,574.45,541.95,509.30,476.49,443.52,410.39,377.09,343.64,310.02,276.24,242.29,208.18,173.90,139.46,104.85,70.07,35.12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
MASAWARA,27.32,26.93,26.53,26.14,25.74,25.34,24.93,24.53,24.12,23.71,23.30,22.89,22.47,22.06,21.64,21.22,20.80,20.37,19.94,19.51,19.08,18.65,18.21,17.78,17.34,16.90,16.45,16.01,15.56,15.11,14.65,14.20,13.74,13.28,12.82,12.36,11.89,11.42,10.95,10.48,10.00,9.53,9.05,8.57,8.08,7.59,7.10,6.61,6.12,5.62,5.12,4.62,4.12,3.61,3.10,2.59,2.08,1.56,1.04,0.52,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,998.98,0.00,0.00,887.95,0.00,0.00,773.22,0.00,0.00,654.67,0.00,0.00,532.17,0.00,0.00,405.59,0.00,0.00,274.80,0.00,0.00,139.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,832.41","1,806.04","1,779.54","1,752.91","1,726.16","1,699.27","1,672.25","1,645.10","1,617.82","1,590.40","1,562.85","1,535.17"

## **Income Statement**

In [46]:
income_statement_index = pd.read_csv("./data_templates/income_statement_index.csv")
income_statement_index

,INCOME_STATEMENT
0,Interest Income
1,Other Income
2,Total Revenue
3,MANAGEMENT EXPENSES
4,STAFF COSTS
5,Salaries
6,Pensions & Statutory Contributions
7,Training
8,Bonus Provision
9,Retrenchments


In [47]:
income_statement_df = income_statement.generate_income_statement_template(
    income_statement_index=income_statement_index,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Calculate Variable Expenses**

In [48]:
variable_expenses = expenses.calculate_variable_expenses(
    variable_inputs_income_statement=variable_inputs_income_statement,
    parameters=parameters,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
variable_expenses

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Travel Costs,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Entertainment,925.21,943.71,962.58,981.84,"1,001.47","1,021.50","1,041.93","1,062.77","1,084.03","1,105.71","1,127.82","1,150.38"
Marketing Costs,"4,490.05","4,579.85","4,671.44","4,764.87","4,860.17","4,957.37","5,056.52","5,157.65","5,260.80","5,366.02","5,473.34","5,582.81"
Group Marketing Costs,"9,898.50","10,096.47","10,298.40","10,504.37","10,714.46","10,928.75","11,147.32","11,370.27","11,597.67","11,829.63","12,066.22","12,307.54"
Rental Costs,"1,199.94","1,223.94","1,248.41","1,273.38","1,298.85","1,324.83","1,351.32","1,378.35","1,405.92","1,434.04","1,462.72","1,491.97"
Subscriptions,585.04,596.74,608.67,620.85,633.26,645.93,658.85,672.03,685.47,699.18,713.16,727.42
Insurance,"2,458.87","2,508.05","2,558.21","2,609.37","2,661.56","2,714.79","2,769.08","2,824.47","2,880.96","2,938.57","2,997.35","3,057.29"
Repairs And Maintenance,"1,116.51","1,138.84","1,161.62","1,184.85","1,208.55","1,232.72","1,257.38","1,282.52","1,308.17","1,334.34","1,361.02","1,388.24"
Utilities,"1,412.25","1,440.49","1,469.30","1,498.69","1,528.66","1,559.23","1,590.42","1,622.23","1,654.67","1,687.76","1,721.52","1,755.95"
Stationery,822.22,838.67,855.44,872.55,890.00,907.80,925.96,944.48,963.37,982.63,"1,002.29","1,022.33"


### **Calculate Change In Provision For Credit Loss**

In [49]:
change_in_provision_for_credit_loss = (
    expenses.calculate_change_in_provision_for_credit_loss(
        provision_for_credit_loss=provision_for_credit_loss_for_all_new_disbursements['total'],
        provision_for_credit_loss_opening_balances=float(
            opening_balances["PROVISION_FOR_CREDIT_LOSS"].iat[0]
        ),
        valuation_date=VALUATION_DATE, 
        months_to_forecast=MONTHS_TO_FORECAST
    )
)

change_in_provision_for_credit_loss

Jan-2023    9,960.00
Feb-2023    2,130.00
Mar-2023    2,190.00
Apr-2023   -1,880.00
May-2023      320.00
Jun-2023    1,620.00
Jul-2023    1,970.00
Aug-2023    4,065.00
Sep-2023    2,775.00
Oct-2023    5,800.00
Nov-2023    2,700.00
Dec-2023        0.00
Name: total, dtype: float64

### **Calculate Business Acquisition**

In [50]:
business_acquisition = expenses.calculate_business_acqusition(
    business_acquisition_percent=parameters.loc['BUSINESS_ACQUISITION_PERCENT'],
    agent_contribution_percent=parameters.loc['AGENT_CONTRIBUTION_PERCENT'],
    consumer_ssb_disbursements=new_disbursements_df['consumer_ssb_disbursements'],
    consumer_pvt_disbursements=new_disbursements_df['consumer_pvt_disbursements'],
)
business_acquisition

Jan-2023   1,620.00
Feb-2023   2,025.00
Mar-2023   2,484.00
Apr-2023   2,760.00
May-2023   3,312.00
Jun-2023   3,366.00
Jul-2023   3,927.00
Aug-2023   4,207.50
Sep-2023   4,275.00
Oct-2023   4,275.00
Nov-2023   4,275.00
Dec-2023   4,275.00
dtype: float64

### **Insert Revenue**

In [51]:
income_statement_df = income_statement.insert_revenue(
    income_statement=income_statement_df,
    interest_income=total_interest_income,
    other_income=other_income_df['total'],
)

income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"9,587.46","55,784.14","105,246.42","155,191.25","187,266.47","215,650.68","238,969.54","261,806.00","298,832.36","343,657.74","411,687.24","481,262.20"
Other Income,"88,739.98","110,958.67","136,366.77","156,709.00","172,038.19","194,702.83","202,140.69","213,807.16","230,057.51","268,132.43","311,807.89","351,419.14"
Total Revenue,"98,327.43","166,742.81","241,613.19","311,900.26","359,304.66","410,353.51","441,110.24","475,613.16","528,889.86","611,790.17","723,495.13","832,681.34"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Static And Variable Inputs**

In [52]:
income_statement_df = income_statement.insert_static_and_variable_inputs(
    income_statement=income_statement_df,
    static_inputs_income_statement=static_inputs_income_statement,
    variable_expenses=variable_expenses,
)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"9,587.46","55,784.14","105,246.42","155,191.25","187,266.47","215,650.68","238,969.54","261,806.00","298,832.36","343,657.74","411,687.24","481,262.20"
Other Income,"88,739.98","110,958.67","136,366.77","156,709.00","172,038.19","194,702.83","202,140.69","213,807.16","230,057.51","268,132.43","311,807.89","351,419.14"
Total Revenue,"98,327.43","166,742.81","241,613.19","311,900.26","359,304.66","410,353.51","441,110.24","475,613.16","528,889.86","611,790.17","723,495.13","832,681.34"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"


### **Insert Salaries and Pensions And Statutory Contributions**

In [53]:
income_statement_df = income_statement.insert_salaries_and_pensions_and_statutory_contributions(
    income_statement=income_statement_df,
    salaries_and_pension_and_statutory_contributions_df = salaries_and_pension_and_statutory_contributions_df,
)

income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"9,587.46","55,784.14","105,246.42","155,191.25","187,266.47","215,650.68","238,969.54","261,806.00","298,832.36","343,657.74","411,687.24","481,262.20"
Other Income,"88,739.98","110,958.67","136,366.77","156,709.00","172,038.19","194,702.83","202,140.69","213,807.16","230,057.51","268,132.43","311,807.89","351,419.14"
Total Revenue,"98,327.43","166,742.81","241,613.19","311,900.26","359,304.66","410,353.51","441,110.24","475,613.16","528,889.86","611,790.17","723,495.13","832,681.34"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,"3,021.76","3,256.91","3,567.04","3,544.32","3,764.70","3,928.29","4,254.32","4,597.96","4,845.04","5,219.92","5,373.28","5,373.28"
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"


### **Insert Depreciation**

In [54]:
income_statement_df = income_statement.insert_depreciation(
    income_statement=income_statement_df, depreciation=depreciations_and_nbvs['dpns']['total'])

### **Insert Change in Provision For Credit Loss**

In [55]:
income_statement_df = income_statement.insert_credit_loss_provision(
    income_statement=income_statement_df, 
    change_in_provisin_for_credit_loss=change_in_provision_for_credit_loss,
)

In [56]:
provision_for_credit_loss  = provision_for_credit_loss_for_all_new_disbursements['total']
provision_for_credit_loss

Jan-2023    9,960.00
Feb-2023   12,090.00
Mar-2023   14,280.00
Apr-2023   12,400.00
May-2023   12,720.00
Jun-2023   14,340.00
Jul-2023   16,310.00
Aug-2023   20,375.00
Sep-2023   23,150.00
Oct-2023   28,950.00
Nov-2023   31,650.00
Dec-2023   31,650.00
Name: total, dtype: float64

### **Insert Business Acquisition**

In [57]:
income_statement_df = income_statement.insert_business_acquisition(
    income_statement=income_statement_df, business_acquisition=business_acquisition)

### **Aggregations**

In [58]:
income_statement_df = income_statement.aggregate_staff_costs(income_statement_df)
income_statement_df = income_statement.aggregate_travel_and_entertainment(income_statement_df)
income_statement_df = income_statement.aggregate_marketing_and_public_relations(income_statement_df)
income_statement_df = income_statement.aggregate_office_costs(income_statement_df)
income_statement_df = income_statement.aggregate_professional_fees(income_statement_df)
income_statement_df = income_statement.aggregate_communication_costs(income_statement_df)
income_statement_df = income_statement.aggregate_motor_vehicle_costs(income_statement_df)
income_statement_df = income_statement.aggregate_other_costs(income_statement_df)
income_statement_df = income_statement.aggregate_investment_income(income_statement_df)
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)

### **Calculating Total Expenses and EBIDTA**

In [59]:
income_statement_df = income_statement.calculate_total_expenses(income_statement_df)
income_statement_df = income_statement.calculate_ebidta(income_statement_df)

### **Insert Finance Cost**

In [60]:
income_statement_df.loc["Finance Costs"] = finance_costs_df.loc["total"]
income_statement_df.loc["Finance Costs"] = income_statement_df.loc[
    "Finance Costs"
].fillna(0)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"9,587.46","55,784.14","105,246.42","155,191.25","187,266.47","215,650.68","238,969.54","261,806.00","298,832.36","343,657.74","411,687.24","481,262.20"
Other Income,"88,739.98","110,958.67","136,366.77","156,709.00","172,038.19","194,702.83","202,140.69","213,807.16","230,057.51","268,132.43","311,807.89","351,419.14"
Total Revenue,"98,327.43","166,742.81","241,613.19","311,900.26","359,304.66","410,353.51","441,110.24","475,613.16","528,889.86","611,790.17","723,495.13","832,681.34"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,"3,021.76","3,256.91","3,567.04","3,544.32","3,764.70","3,928.29","4,254.32","4,597.96","4,845.04","5,219.92","5,373.28","5,373.28"
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"


### **Aggregate Finance Cost**

In [61]:
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)

### **Calculate Profit Before Tax**

In [62]:
income_statement_df = income_statement.calculate_profit_before_tax(income_statement_df)

### **Calculate Tax**

In [63]:
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"9,587.46","55,784.14","105,246.42","155,191.25","187,266.47","215,650.68","238,969.54","261,806.00","298,832.36","343,657.74","411,687.24","481,262.20"
Other Income,"88,739.98","110,958.67","136,366.77","156,709.00","172,038.19","194,702.83","202,140.69","213,807.16","230,057.51","268,132.43","311,807.89","351,419.14"
Total Revenue,"98,327.43","166,742.81","241,613.19","311,900.26","359,304.66","410,353.51","441,110.24","475,613.16","528,889.86","611,790.17","723,495.13","832,681.34"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,"3,021.76","3,256.91","3,567.04","3,544.32","3,764.70","3,928.29","4,254.32","4,597.96","4,845.04","5,219.92","5,373.28","5,373.28"
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"


In [64]:
income_statement_df = income_statement.calculate_tax(income_statement=income_statement_df,tax_rate=parameters.loc['TAX_RATE'])
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"9,587.46","55,784.14","105,246.42","155,191.25","187,266.47","215,650.68","238,969.54","261,806.00","298,832.36","343,657.74","411,687.24","481,262.20"
Other Income,"88,739.98","110,958.67","136,366.77","156,709.00","172,038.19","194,702.83","202,140.69","213,807.16","230,057.51","268,132.43","311,807.89","351,419.14"
Total Revenue,"98,327.43","166,742.81","241,613.19","311,900.26","359,304.66","410,353.51","441,110.24","475,613.16","528,889.86","611,790.17","723,495.13","832,681.34"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,"3,021.76","3,256.91","3,567.04","3,544.32","3,764.70","3,928.29","4,254.32","4,597.96","4,845.04","5,219.92","5,373.28","5,373.28"
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"


## **Direct Cashflow**

In [65]:
direct_cashflow_df = direct_cashflow.generate_direct_cashflow_template(
    valuation_date=VALUATION_DATE, months_to_forecast=MONTHS_TO_FORECAST
)

direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Receipts From Receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sale Of Other Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Issue Of Equity And Intercompany Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Available Direct Cashflow Items**

#### **From Parameters**

In [66]:
direct_cashflow_df.loc[
    "Receipts From Receivables"
] = helper.change_period_index_to_strftime(
    helper.add_series(
        [
            parameters.loc["RECEIPTS_FROM_OTHER_RECEIVABLES"],
            parameters.loc["RECEIPTS_FROM_TRADE_RECEIVABLES"],
            parameters.loc["RECEIPTS_FROM_INTERGROUP_RECEIVABLES"],
        ]
    )
)

direct_cashflow_df.loc[
    "Purchase Of Inventory"
] = helper.change_period_index_to_strftime(parameters.loc["NEW_INVENTORY"])
direct_cashflow_df.loc[
    "Payments To Payables"
] = -helper.change_period_index_to_strftime(
    helper.add_series(
        [
            parameters.loc["PAYMENTS_TO_TRADE_PAYABLES"],
            parameters.loc["PAYMENTS_TO_OTHER_PAYABLES"],
        ]
    )
)

direct_cashflow_df.loc["Dividend Paid"] = -helper.change_period_index_to_strftime(
    parameters.loc["DIVIDEND_PAID"]
)

#### **From Calculations/Income Statement**

In [67]:
direct_cashflow_df.loc['Interest Income'] = income_statement_df.loc['Interest Income']
direct_cashflow_df.loc['Other Income'] = income_statement_df.loc['Other Income']
direct_cashflow_df.loc['Interest Expense'] = -income_statement_df.loc['Finance Costs']
direct_cashflow_df.loc['Disbursements'] = -helper.change_period_index_to_strftime(new_disbursements_df['total'])

In [68]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,"9,587.46","55,784.14","105,246.42","155,191.25","187,266.47","215,650.68","238,969.54","261,806.00","298,832.36","343,657.74","411,687.24","481,262.20"
Other Income,"88,739.98","110,958.67","136,366.77","156,709.00","172,038.19","194,702.83","202,140.69","213,807.16","230,057.51","268,132.43","311,807.89","351,419.14"
Receipts From Receivables,0.00,0.00,0.00,0.00,"1,187,638.92",0.00,0.00,0.00,0.00,0.00,0.00,0.00
Sale Of Other Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Issue Of Equity And Intercompany Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Equity and Intercompany Loans**

In [69]:
direct_cashflow_df = direct_cashflow.add_equity_and_intercompany_loans(
    parameters=parameters, direct_cashflow_df=direct_cashflow_df
)

### **Other Assets**


In [70]:
direct_cashflow_df = direct_cashflow.add_other_assets(
    parameters=parameters, direct_cashflow_df=direct_cashflow_df
)

### **Tax Paid**

In [71]:
tax_schedule = direct_cashflow.generate_tax_schedule(
    taxation=income_statement_df.loc["Taxation"],
    opening_balance=opening_balances["DEFERED_TAXATION"].iat[0],
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)
tax_schedule

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,0,0,0,0,0,0,0,0,0,0,0,0
Tax Charged,0,0,0,0,0,0,0,0,0,0,0,0
Tax Paid,0,0,0,0,0,0,0,0,0,0,0,0
Closing Balance,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
direct_cashflow_df.loc['Tax Paid'] = tax_schedule.loc['Tax Paid']

### **Calculate Operating Expenses**

In [73]:
operating_expenses = direct_cashflow.calculate_operating_expenses(income_statement=income_statement_df)
operating_expenses

Jan-2023    8,093,483.48
Feb-2023    8,261,284.52
Mar-2023    8,427,915.55
Apr-2023    8,597,687.06
May-2023    8,772,814.47
Jun-2023    8,950,566.95
Jul-2023    9,143,290.32
Aug-2023    9,325,712.67
Sep-2023    9,514,201.58
Oct-2023    9,708,858.41
Nov-2023    9,903,147.43
Dec-2023   10,097,779.36
dtype: object

### **Insert Operating Expenses**

In [74]:
direct_cashflow_df.loc['Operating Expenses']  = -operating_expenses
direct_cashflow_df


,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,"9,587.46","55,784.14","105,246.42","155,191.25","187,266.47","215,650.68","238,969.54","261,806.00","298,832.36","343,657.74","411,687.24","481,262.20"
Other Income,"88,739.98","110,958.67","136,366.77","156,709.00","172,038.19","194,702.83","202,140.69","213,807.16","230,057.51","268,132.43","311,807.89","351,419.14"
Receipts From Receivables,0.00,0.00,0.00,0.00,"1,187,638.92",0.00,0.00,0.00,0.00,0.00,0.00,0.00
Sale Of Other Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,0.00,0.00,0.00,0.00,0.00,0.00,61.99,98.62,775.08,0.00,0.00,0.00


### **Calculate Capital Expenses**

In [75]:
capital_expenses = direct_cashflow.calculate_capital_expenses(
    details_of_new_assets=details_of_new_assets,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)

capital_expenses

Jan-2023    0
Feb-2023    0
Mar-2023    0
Apr-2023    0
May-2023    0
Jun-2023    0
Jul-2023    0
Aug-2023    0
Sep-2023    0
Oct-2023    0
Nov-2023    0
Dec-2023    0
Name: book_value, dtype: object

### **Insert Capital Expenses**

In [76]:
direct_cashflow_df.loc['Capital Expenses'] = -capital_expenses

### **Calculate Direct Cashflow Borrowings**

In [77]:
details_of_new_long_term_borrowing

,effective_date,institution,nominal_amount,interest_rate,tenure,repayment_frequency,frequency,method
0,31/12/2023,ZIMNAT LIFE ASSURANCE,376455,0.06,60,Monthly (P+I),12,reducing_balance
1,06/05/2024,MASAWARA,5613,0.06,60,Monthly (P+I),12,reducing_balance


In [78]:
details_of_new_long_term_borrowing = helper.columns_to_snake_case(details_of_long_term_borrowings)
details_of_new_short_term_borrowing = helper.columns_to_snake_case(details_of_short_term_borrowings)

In [80]:
details_of_long_term_borrowings

,effective_date,institution,nominal_amount,interest_rate,tenure,repayment_frequency,frequency,method
0,2022-12-31,ZIMNAT LIFE ASSURANCE,376455,0.06,60,Monthly (P+I),12,reducing_balance
1,2022-05-06,MASAWARA,5613,0.06,60,Monthly (P+I),12,reducing_balance
2,2022-12-31,Grandre,160000,0.06,36,Quaterly (I+Bullet P),4,straight_line
3,2022-12-31,MINERVA,15000,0.08,36,Bullet (P+I),0,straight_line
4,2022-09-10,ZAM-CARGO,364000,0.05,36,Annually(I) Principal rollover,12,straight_line
5,2023-04-05,ZIMNAT LIFE ASSURANCE,30000,0.14,24,Monthly (P+I),4,reducing_balance
6,2023-12-31,ZIMNAT LIFE ASSURANCE,376455,0.06,60,Monthly (P+I),12,reducing_balance
7,2024-05-06,MASAWARA,5613,0.06,60,Monthly (P+I),12,reducing_balance


In [81]:
details_of_short_term_borrowings

,effective_date,institution,nominal_amount,interest_rate,tenure,repayment_frequency,frequency,method
0,2023-04-24,FCB,"120,383.84",0.18,3,Bullet (P+I),0,straight_line
1,2023-03-02,ZIMNAT LIFE ASSURANCE,"50,000.00",0.14,6,Monthly (P+I),12,reducing_balance
2,2022-12-31,Zimnat Life Assurance,"20,000.00",0.08,12,Annually(I) Principal rollover,1,straight_line
3,2022-12-31,ZLA-Cargo,"124,575.00",0.10,12,Annually(I) Principal rollover,1,straight_line
4,2022-12-31,ZIMNAT LION INSURANCE,"50,000.00",0.08,12,Bi-annual(I)Principal at maturity,2,straight_line
5,2022-12-31,ZIMNAT LION INSURANCE,"92,800.00",0.06,12,Bi-annual(I)Principal at maturity,2,straight_line
6,2022-12-31,Grandre,"100,000.00",0.12,12,Quaterly (I+Bullet P),4,straight_line
7,2023-04-23,FCB,"18,429.93",1.50,12,Bullet (P+I),0,straight_line
8,2022-12-17,NURTURE,"500,000.00",0.20,12,Quaterly (I+Bullet P),4,straight_line
9,2023-01-26,NMB,"450,000.00",0.18,12,Quaterly (I+Bullet P),4,straight_line


In [79]:
long_and_short_term_borrowing_df = (
    direct_cashflow.calculate_long_and_short_term_borrowing_for_direct_cashflow(
        details_of_new_long_term_borrowing=details_of_new_long_term_borrowing,
        details_of_new_short_term_borrowing=details_of_new_short_term_borrowing,
        valuation_date=VALUATION_DATE,
        months_to_forecast=MONTHS_TO_FORECAST,
    )
)

long_and_short_term_borrowing_df

,long_term_borrowing,short_term_borrowing,total
Jan-2023,0,"450,000.00","450,000.00"
Feb-2023,0,0.00,0.00
Mar-2023,0,"105,000.00","105,000.00"
Apr-2023,30000,"138,813.77","168,813.77"
May-2023,0,0.00,0.00
Jun-2023,0,0.00,0.00
Jul-2023,0,0.00,0.00
Aug-2023,0,0.00,0.00
Sep-2023,0,0.00,0.00
Oct-2023,0,0.00,0.00


### **Insert Direct Cashflow Borrowing**

In [109]:
direct_cashflow_df.loc['Short Term Borrowing'] = long_and_short_term_borrowing_df['short_term_borrowing']
direct_cashflow_df.loc['Long Term Borrowing'] = long_and_short_term_borrowing_df['long_term_borrowing']

In [110]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,"450,000.00",0.00,"105,000.00","138,813.77",0.00,0.00,0.00,0.00,0.00,0.00,0.00,"376,455.00"
Long Term Borrowing,0.00,0.00,0.00,"30,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,"376,455.00"
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,"9,587.46","55,073.81","104,798.13","155,012.37","187,137.16","214,649.70","238,614.26","261,403.79","299,357.24","343,344.04","410,445.43","480,958.24"
Other Income,"76,240.63","84,598.56","94,278.75","101,537.79","103,261.42","112,062.63","112,708.96","115,555.28","119,507.01","134,434.02","153,949.00","170,787.47"
Receipts From Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Purchase Of Inventory,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,"1,000.00",0.00,0.00,0.00,0.00,"1,000.00",0.00,"2,000.00",0.00,100.00,0.00,0.00


### **Calculate Capital Repayments**

In [111]:
capital_repayment = helper.add_series(
[existing_loans_schedules['capital_repayments'].sum()
    , capital_repayment_new_disbursements_df['total']]
)

capital_repayment

Jan-2019      629.39
Feb-2019    3,245.44
Mar-2019    8,021.09
Apr-2019    9,315.64
May-2019   12,737.70
              ...   
Jun-2027    4,173.62
Jul-2027    2,747.85
Aug-2027    1,318.57
Sep-2027    1,321.82
Oct-2027    1,325.08
Length: 106, dtype: float64

### **Insert Capital Repayment**

In [112]:
direct_cashflow_df.loc['Capital Repayment'] = capital_repayment

### **Calculate Capital Repayments on Borrowings**


In [113]:
capital_repayment_borrowings_df

,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025,Jul-2025,Aug-2025,Sep-2025,Oct-2025,Nov-2025,Dec-2025,Jan-2026,Feb-2026,Mar-2026,Apr-2026,May-2026,Jun-2026,Jul-2026,Aug-2026,Sep-2026,Oct-2026,Nov-2026,Dec-2026,Jan-2027,Feb-2027,Mar-2027,Apr-2027,May-2027,Jun-2027,Jul-2027,Aug-2027,Sep-2027,Oct-2027,Nov-2027,Dec-2027,Jan-2028,Feb-2028,Mar-2028,Apr-2028,May-2028,Jun-2028,Jul-2028,Aug-2028,Sep-2028,Oct-2028,Nov-2028,Dec-2028,Jan-2029,Feb-2029,Mar-2029,Apr-2029
Grandre,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"160,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
MINERVA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"15,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZAM-CARGO,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"364,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,417.73","5,444.10","5,470.60","5,497.23","5,523.98","5,550.87","5,577.89","5,605.04","5,632.32","5,659.74","5,687.29","5,714.97","5,742.79","5,770.74","5,798.83","5,827.06","5,855.42","5,883.92","5,912.56","5,941.34","5,970.26","5,999.32","6,028.53","6,057.87","6,087.36","6,116.99","6,146.76","6,176.68","6,206.75","6,236.96","6,267.32","6,297.82","6,328.48","6,359.28","6,390.24","6,421.34","6,452.60","6,484.01","6,515.57","6,547.28","6,579.15","6,611.18","6,643.36","6,675.69","6,708.19","6,740.84","6,773.65","6,806.62","6,839.76","6,873.05","6,906.50","6,940.12","6,973.90","7,007.85","7,041.96","7,076.24","7,110.68","7,145.29","7,180.07","7,215.02",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
MASAWARA,80.78,81.17,81.57,81.96,82.36,82.76,83.17,83.57,83.98,84.39,84.80,85.21,85.63,86.04,86.46,86.88,87.31,87.73,88.16,88.59,89.02,89.45,89.89,90.32,90.76,91.21,91.65,92.10,92.54,92.99,93.45,93.90,94.36,94.82,95.28,95.74,96.21,96.68,97.15,97.62,98.10,98.57,99.05,99.54,100.02,100.51,101.00,101.49,101.98,102.48,102.98,103.48,103.98,104.49,105.00,105.51,106.02,106.54,107.06,107.58,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"3,334.39",0.00,0.00,"3,445.43",0.00,0.00,"3,560.16",0.00,0.00,"3,678.71",0.00,0.00,"3,801.21",0.00,0.00,"3,927.79",0.00,0.00,"4,058.58",0.00,0.00,"4,193.73",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,417.73","5,4

### **Insert Capital Repayment on New Borrowings**

In [114]:
direct_cashflow_df.loc[
    "Capital Repayment On Borrowings"
] = -capital_repayment_borrowings_df.loc["total"]

direct_cashflow_df.loc["Capital Repayment On Borrowings"] = direct_cashflow_df.loc[
    "Capital Repayment On Borrowings"
].fillna(0)

### **Calculate Direct Cashflow Totals**

In [115]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,"450,000.00",0.00,"105,000.00","138,813.77",0.00,0.00,0.00,0.00,0.00,0.00,0.00,"376,455.00"
Long Term Borrowing,0.00,0.00,0.00,"30,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,"376,455.00"
Capital Repayment,"300,976.91","368,846.64","466,412.98","596,412.00","619,558.56","737,349.66","828,216.30","871,911.38","901,876.73","908,459.26","976,571.61","1,059,020.23"
Interest Income,"9,587.46","55,073.81","104,798.13","155,012.37","187,137.16","214,649.70","238,614.26","261,403.79","299,357.24","343,344.04","410,445.43","480,958.24"
Other Income,"76,240.63","84,598.56","94,278.75","101,537.79","103,261.42","112,062.63","112,708.96","115,555.28","119,507.01","134,434.02","153,949.00","170,787.47"
Receipts From Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Purchase Of Inventory,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,"1,000.00",0.00,0.00,0.00,0.00,"1,000.00",0.00,"2,000.00",0.00,100.00,0.00,0.00


0

In [117]:
direct_cashflow_df.loc["Total Cash Inflows"] = direct_cashflow_df.iloc[
    direct_cashflow_df.index.get_loc("CASH INFLOWS")
    + 1 : direct_cashflow_df.index.get_loc("Total Cash Inflows")
].sum()


direct_cashflow_df.loc["Total Cash Outflows"] = direct_cashflow_df.iloc[
    direct_cashflow_df.index.get_loc("CASH OUTFLOWS")
    + 1 : direct_cashflow_df.index.get_loc("Total Cash Outflows")
].sum()

direct_cashflow_df.loc["Net Increase/Decrease In Cash"] = (
    direct_cashflow_df.loc["Total Cash Inflows"]
    + direct_cashflow_df.loc["Total Cash Outflows"]
)

### **Calculate Opening and Closing Balances**

In [120]:
depreciations_and_nbvs['dpns'].sum(axis = 1)

Jan-2023   5,959,732.44
Feb-2023   5,959,732.44
Mar-2023   5,959,732.44
Apr-2023   5,959,732.44
May-2023   5,959,732.44
Jun-2023   5,959,732.44
Jul-2023   5,959,732.44
Aug-2023   5,959,732.44
Sep-2023   5,959,732.44
Oct-2023   5,959,732.44
Nov-2023   5,958,732.54
Dec-2023   5,958,048.17
dtype: float64

In [124]:
depreciations_and_nbvs['nbvs'].sum(axis = 1)

Jan-2023   295,484,695.90
Feb-2023   289,524,963.46
Mar-2023   283,565,231.01
Apr-2023   277,605,498.57
May-2023   271,645,766.13
Jun-2023   265,686,033.69
Jul-2023   259,726,301.25
Aug-2023   253,766,568.80
Sep-2023   247,806,836.36
Oct-2023   241,847,103.92
Nov-2023   235,894,478.54
Dec-2023   229,941,221.05
dtype: float64

In [353]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"376,455.00"
Long Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"376,455.00"
Capital Repayment,"148,576.53","198,996.23","288,929.08","386,512.45","403,648.63","472,575.34","536,371.91","566,702.29","566,110.55","614,574.74","715,122.21","845,259.64"
Interest Income,"3,925.05","48,488.03","97,752.56","147,475.18","179,319.33","203,927.28","227,565.56","250,100.30","287,972.79","334,165.97","403,169.47","475,340.40"
Other Income,"31,950.68","41,755.74","53,176.03","61,427.87","66,054.29","76,736.59","78,839.31","84,887.84","90,764.26","109,386.24","132,088.80","152,784.10"
Receipts From Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Purchase Of Inventory,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,"1,000.00",0.00,0.00,0.00,0.00,"1,000.00",0.00,"2,000.00",0.00,100.00,0.00,0.00
Purchase Of Other Assets,0.00,0.00,0.00,0.00,"10,000.00","1,000.00","1,000.00",0.00,"5,000.00",0.00,0.00,"10,000.00"


In [354]:
direct_cashflow_df= direct_cashflow.calculate_opening_and_closing_balances_for_direct_cashflows(
direct_cashflow=direct_cashflow_df, 
cash_on_hand_opening_balance= opening_balances['CASH_ON_HAND'].iat[0]
)

direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"376,455.00"
Long Term Borrowing,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"376,455.00"
Capital Repayment,"148,576.53","198,996.23","288,929.08","386,512.45","403,648.63","472,575.34","536,371.91","566,702.29","566,110.55","614,574.74","715,122.21","845,259.64"
Interest Income,"3,925.05","48,488.03","97,752.56","147,475.18","179,319.33","203,927.28","227,565.56","250,100.30","287,972.79","334,165.97","403,169.47","475,340.40"
Other Income,"31,950.68","41,755.74","53,176.03","61,427.87","66,054.29","76,736.59","78,839.31","84,887.84","90,764.26","109,386.24","132,088.80","152,784.10"
Receipts From Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Purchase Of Inventory,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,"1,000.00",0.00,0.00,0.00,0.00,"1,000.00",0.00,"2,000.00",0.00,100.00,0.00,0.00
Purchase Of Other Assets,0.00,0.00,0.00,0.00,"10,000.00","1,000.00","1,000.00",0.00,"5,000.00",0.00,0.00,"10,000.00"


### **Calculate IMTT (2% Tax)**

In [355]:
income_statement_df.loc['2% Taxation']  = -direct_cashflow_df.loc['Total Cash Outflows']*IMTT

## **Calculate Profit and Loss for the Period**


In [356]:
income_statement_df = income_statement.calculate_profit_or_loss_for_period(income_statement_df)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"3,925.05","48,488.03","97,752.56","147,475.18","179,319.33","203,927.28","227,565.56","250,100.30","287,972.79","334,165.97","403,169.47","475,340.40"
Other Income,"31,950.68","41,755.74","53,176.03","61,427.87","66,054.29","76,736.59","78,839.31","84,887.84","90,764.26","109,386.24","132,088.80","152,784.10"
Total Revenue,"35,875.74","90,243.77","150,928.59","208,903.05","245,373.62","280,663.87","306,404.87","334,988.14","378,737.05","443,552.21","535,258.27","628,124.50"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,"3,021.76","3,256.91","3,567.04","3,544.32","3,764.70","3,928.29","4,254.32","4,597.96","4,845.04","5,219.92","5,373.28","5,373.28"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


## **Loan Book**

In [357]:
loan_book_df = loan_book.generate_loan_book_template(
    valuation_date=VALUATION_DATE, months_to_forecast=MONTHS_TO_FORECAST
)
loan_book_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Repayments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Loan Book Items**

In [360]:
capital_repayment_existing_loans = existing_loans_schedules['capital_repayments'].sum()
capital_repayment_existing_loans

Jan-2019       629.39
Feb-2019     2,647.28
Mar-2019     6,496.81
Apr-2019     7,379.95
May-2019    10,060.68
Jun-2019     9,693.15
Jul-2019     8,750.38
Aug-2019     7,877.35
Sep-2019     4,952.02
Oct-2019     6,591.01
Nov-2019    25,022.59
Dec-2019     7,319.56
Jan-2020    13,098.72
Feb-2020    18,520.44
Mar-2020    19,964.67
Apr-2020    17,549.39
May-2020    20,953.54
Jun-2020    24,450.50
Jul-2020    47,511.45
Aug-2020    51,818.55
Sep-2020    60,373.24
Oct-2020    70,223.94
Nov-2020    76,587.44
Dec-2020    52,126.81
Jan-2021    49,732.92
Feb-2021    53,165.18
Mar-2021    59,596.72
Apr-2021    66,791.07
May-2021    72,085.81
Jun-2021    84,410.40
Jul-2021    85,129.29
Aug-2021    89,500.79
Sep-2021    90,692.20
Oct-2021    99,318.78
Nov-2021   113,987.67
Dec-2021   106,737.01
Jan-2022   116,561.30
Feb-2022   127,686.07
Mar-2022   131,165.92
Apr-2022   137,144.82
May-2022   131,485.97
Jun-2022   138,804.89
Jul-2022   128,644.70
Aug-2022   144,281.89
Sep-2022   144,552.10
Oct-2022  

In [361]:
total_capital_repayments = loan_book.aggregate_new_and_existing_loans_capital_repayments(
    capital_repayments_new_disbursements_df = capital_repayment_new_disbursements_df,
    capital_repayments_existing_loans = capital_repayment_existing_loans,
    valuation_date = VALUATION_DATE,
    months_to_forecast = MONTHS_TO_FORECAST,
)

total_capital_repayments

Jan-2023   148,576.53
Feb-2023   198,996.23
Mar-2023   288,929.08
Apr-2023   386,512.45
May-2023   403,648.63
Jun-2023   472,575.34
Jul-2023   536,371.91
Aug-2023   566,702.29
Sep-2023   566,110.55
Oct-2023   614,574.74
Nov-2023   715,122.21
Dec-2023   845,259.64
dtype: float64

In [362]:
loan_book_df = loan_book.insert_loan_book_items(
    loan_book=loan_book_df,
    opening_balance_on_loan_book=float(opening_balances["LOAN_BOOK"].iat[0]),
    total_interest_income=total_interest_income,
    total_capital_repayments=total_capital_repayments,
    disbursements=helper.change_period_index_to_strftime(new_disbursements_df["total"]),
)

loan_book_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,"2,800,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,"490,000.00","592,000.00","706,000.00","610,000.00","622,000.00","712,000.00","808,000.00","1,010,000.00","1,160,000.00","1,450,000.00","1,585,000.00","1,585,000.00"
Repayments,"-152,501.59","-247,484.25","-386,681.64","-533,987.63","-582,967.96","-676,502.62","-763,937.46","-816,802.59","-854,083.34","-948,740.71","-1,118,291.68","-1,320,600.04"
Interest Income,"3,925.05","48,488.03","97,752.56","147,475.18","179,319.33","203,927.28","227,565.56","250,100.30","287,972.79","334,165.97","403,169.47","475,340.40"
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Calculate Opening and Closing Balances for Loan Book**

In [363]:
loan_book_df = helper.calculate_opening_and_closing_balances(loan_book_df)
loan_book_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,"2,800,000.00","3,141,423.47","3,534,427.24","3,951,498.16","4,174,985.71","4,393,337.08","4,632,761.74","4,904,389.83","5,347,687.54","5,941,576.99","6,777,002.25","7,646,880.04"
New Disbursements,"490,000.00","592,000.00","706,000.00","610,000.00","622,000.00","712,000.00","808,000.00","1,010,000.00","1,160,000.00","1,450,000.00","1,585,000.00","1,585,000.00"
Repayments,"-152,501.59","-247,484.25","-386,681.64","-533,987.63","-582,967.96","-676,502.62","-763,937.46","-816,802.59","-854,083.34","-948,740.71","-1,118,291.68","-1,320,600.04"
Interest Income,"3,925.05","48,488.03","97,752.56","147,475.18","179,319.33","203,927.28","227,565.56","250,100.30","287,972.79","334,165.97","403,169.47","475,340.40"
Closing Balance,"3,141,423.47","3,534,427.24","3,951,498.16","4,174,985.71","4,393,337.08","4,632,761.74","4,904,389.83","5,347,687.54","5,941,576.99","6,777,002.25","7,646,880.04","8,386,620.39"


## **Balance Sheet**


In [364]:
balance_sheet_df = balance_sheet.generate_balance_sheet_template(
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)

balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Intangible Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Subsidiaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Associates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment Properties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Equity Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Money Market Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Available Balance Sheet Items**

In [365]:
depreciations_and_nbvs.keys()

dict_keys(['nbvs', 'dpns'])

In [366]:
balance_sheet_df.loc["Property Plant And Equipment"] = depreciations_and_nbvs["nbvs"][
    "total"
]

balance_sheet_df.loc["Loan Book"] = loan_book_df.loc["Closing Balance"]

balance_sheet_df.loc["Cash On Hand"] = direct_cashflow_df.loc["Closing Balance"]

balance_sheet_df.loc[
    "Provisions"
] = provision_for_credit_loss_for_all_new_disbursements["total"]


balance_sheet_df.loc["Provision For Taxation"] = helper.change_period_index_to_strftime(
    parameters.loc["PROVISION_FOR_TAX"]
)


balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"147,742,347.95","144,762,481.73","141,782,615.51","138,802,749.29","135,822,883.07","132,843,016.84","129,863,150.62","126,883,284.40","123,903,418.18","120,923,551.96","117,947,239.27","114,970,610.52"
Intangible Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Subsidiaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Associates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment Properties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Equity Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Money Market Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Generate Borrowings Schedules**

In [367]:
short_term_loans_schedules = balance_sheet.calculate_short_term_loans_schedules(
    long_and_short_term_borrowing_df=long_and_short_term_borrowing_df,
    capital_repayment_on_borrowings_df=short_term_borrowings_schedules['capital_payments'].sum(),
    opening_balances=opening_balances,
    valuation_date=VALUATION_DATE, 
    months_to_forecast=MONTHS_TO_FORECAST
)

short_term_loans_schedules

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,50000,50000,50000,"1,201.06",741.13,371.18,"-52,941.78","-53,128.77","-53,222.78","-102,372.78","-102,372.78","-102,372.78"
Borrowings,0,0,0,0,0,0,0,0,0,0,0,376455
Repayments,NaN,NaN,"-48,798.94",-459.93,-369.94,"-53,312.97",-186.99,-94.01,"-49,150.00",NaN,NaN,"-42,091.50"
Closing Balance,50000,50000,"1,201.06",741.13,371.18,"-52,941.78","-53,128.77","-53,222.78","-102,372.78","-102,372.78","-102,372.78","231,990.72"


In [368]:
capital_repayment_borrowings_df

,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025,Jul-2025,Aug-2025,Sep-2025,Oct-2025,Nov-2025,Dec-2025,Jan-2026,Feb-2026,Mar-2026,Apr-2026,May-2026,Jun-2026,Jul-2026,Aug-2026,Sep-2026,Oct-2026,Nov-2026,Dec-2026,Jan-2027,Feb-2027,Mar-2027,Apr-2027,May-2027,Jun-2027,Jul-2027,Aug-2027,Sep-2027,Oct-2027,Nov-2027,Dec-2027,Jan-2028,Feb-2028,Mar-2028,Apr-2028,May-2028,Jun-2028,Jul-2028,Aug-2028,Sep-2028,Oct-2028,Nov-2028,Dec-2028,Jan-2029,Feb-2029,Mar-2029,Apr-2029
Grandre,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"160,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
MINERVA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"15,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZAM-CARGO,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"364,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,417.73","5,444.10","5,470.60","5,497.23","5,523.98","5,550.87","5,577.89","5,605.04","5,632.32","5,659.74","5,687.29","5,714.97","5,742.79","5,770.74","5,798.83","5,827.06","5,855.42","5,883.92","5,912.56","5,941.34","5,970.26","5,999.32","6,028.53","6,057.87","6,087.36","6,116.99","6,146.76","6,176.68","6,206.75","6,236.96","6,267.32","6,297.82","6,328.48","6,359.28","6,390.24","6,421.34","6,452.60","6,484.01","6,515.57","6,547.28","6,579.15","6,611.18","6,643.36","6,675.69","6,708.19","6,740.84","6,773.65","6,806.62","6,839.76","6,873.05","6,906.50","6,940.12","6,973.90","7,007.85","7,041.96","7,076.24","7,110.68","7,145.29","7,180.07","7,215.02",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
MASAWARA,80.78,81.17,81.57,81.96,82.36,82.76,83.17,83.57,83.98,84.39,84.80,85.21,85.63,86.04,86.46,86.88,87.31,87.73,88.16,88.59,89.02,89.45,89.89,90.32,90.76,91.21,91.65,92.10,92.54,92.99,93.45,93.90,94.36,94.82,95.28,95.74,96.21,96.68,97.15,97.62,98.10,98.57,99.05,99.54,100.02,100.51,101.00,101.49,101.98,102.48,102.98,103.48,103.98,104.49,105.00,105.51,106.02,106.54,107.06,107.58,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"3,334.39",0.00,0.00,"3,445.43",0.00,0.00,"3,560.16",0.00,0.00,"3,678.71",0.00,0.00,"3,801.21",0.00,0.00,"3,927.79",0.00,0.00,"4,058.58",0.00,0.00,"4,193.73",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,417.73","5,4

In [369]:
long_and_short_term_borrowing_df

,long_term_borrowing,short_term_borrowing,total
Jan-2023,0,0,0
Feb-2023,0,0,0
Mar-2023,0,0,0
Apr-2023,0,0,0
May-2023,0,0,0
Jun-2023,0,0,0
Jul-2023,0,0,0
Aug-2023,0,0,0
Sep-2023,0,0,0
Oct-2023,0,0,0


In [414]:
long_term_loans_schedules = balance_sheet.calculate_long_term_loans_schedules(
    long_and_short_term_borrowing_df=long_and_short_term_borrowing_df,
    capital_repayment_on_borrowings_df=long_term_borrowings_schedules['interest_payments'].sum(),
    opening_balances=opening_balances,
    valuation_date=VALUATION_DATE, 
    months_to_forecast=MONTHS_TO_FORECAST
)

long_term_loans_schedules

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,200000,"196,626.80","193,280.38","187,560.86","184,268.39","181,003.09","174,366.12","171,155.56","167,972.58","163,046.01","161,435.24","159,852.45"
Borrowings,0,0,0,0,0,0,0,0,0,0,0,376455
Repayments,"-3,373.20","-3,346.42","-5,719.51","-3,292.47","-3,265.30","-6,636.98","-3,210.56","-3,182.98","-4,926.56","-1,610.77","-1,582.79","-4,727.90"
Closing Balance,"196,626.80","193,280.38","187,560.86","184,268.39","181,003.09","174,366.12","171,155.56","167,972.58","163,046.01","161,435.24","159,852.45","531,579.55"


### **Insert Borrowings and Loans**


In [371]:
balance_sheet_df.loc["Loans"] = long_term_loans_schedules.loc['Closing Balance']
balance_sheet_df.loc["Borrowings"] = short_term_loans_schedules.loc['Closing Balance']

### **Generate Receivables and Payables Schedules**

In [415]:
trade_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["TRADE_RECEIVABLES"].iat[0],
    receipts_from_receivables=parameters.loc["RECEIPTS_FROM_TRADE_RECEIVABLES"],
    new_receivables=parameters.loc["NEW_TRADE_RECEIVABLES"],
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)

other_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["OTHER_RECEIVABLES"].iat[0],
    receipts_from_receivables=parameters.loc["RECEIPTS_FROM_OTHER_RECEIVABLES"],
    new_receivables=parameters.loc["NEW_OTHER_RECEIVABLES"],
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)

intergroup_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["INTERGROUP_RECEIVABLES"].iat[0],
    receipts_from_receivables=parameters.loc["RECEIPTS_FROM_INTERGROUP_RECEIVABLES"],
    new_receivables=parameters.loc["NEW_INTERGROUP_RECEIVABLES"],
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)

trade_payables_schedule = balance_sheet.generate_payables_schedule(
    opening_payables=opening_balances["TRADE_PAYABLES"].iat[0],
    payments_to_payables=parameters.loc["PAYMENTS_TO_TRADE_PAYABLES"],
    new_payables=parameters.loc["NEW_TRADE_PAYABLES"],
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)

other_payables_schedule = balance_sheet.generate_payables_schedule(
    opening_payables=opening_balances["OTHER_PAYABLES"].iat[0],
    payments_to_payables=parameters.loc["PAYMENTS_TO_OTHER_PAYABLES"],
    new_payables=parameters.loc["NEW_OTHER_PAYABLES"],
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)


balance_sheet_df.loc["Trade Payables"] = trade_payables_schedule.loc["Closing Balance"]
balance_sheet_df.loc["Other Payables"] = other_payables_schedule.loc["Closing Balance"]

balance_sheet_df.loc["Trade Receivables"] = trade_receivables_schedule.loc[
    "Closing Balance"
]

balance_sheet_df.loc["Other Receivables"] = other_receivables_schedule.loc[
    "Closing Balance"
]

balance_sheet_df.loc["Intergroup Receivables"] = intergroup_receivables_schedule.loc[
    "Closing Balance"
]


balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"147,742,347.95","144,762,481.73","141,782,615.51","138,802,749.29","135,822,883.07","132,843,016.84","129,863,150.62","126,883,284.40","123,903,418.18","120,923,551.96","117,947,239.27","114,970,610.52"
Intangible Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Investment In Subsidiaries,0.00,"-1,000.00","-1,000.00","-1,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00"
Investment In Associates,"10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","11,000.00","11,000.00","11,000.00","11,000.00","11,000.00","11,000.00"
Investment Properties,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,000.00","5,000.00","5,000.00","5,000.00"
Equity Investments,"20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","21,000.00","21,000.00","21,000.00","21,000.00","21,000.00","21,000.00","31,000.00"
Long Term Money Market Investments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [416]:
balance_sheet_df.loc['Deferred Taxation'] = tax_schedule.loc['Closing Balance']

In [417]:
inventories_schedule = balance_sheet.generate_inventories_schedule(
    opening_inventories=opening_balances["INVENTORIES"].iat[0],
    new_inventories=parameters.loc["NEW_INVENTORY"],
    inventories_used=parameters.loc["INVENTORY_USED"],
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
)

inventories_schedule

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,0,0,0,0,0,0,0,0,0,0,0,0
New Inventories,0,0,0,0,0,0,0,0,0,0,0,0
Inventories Used,0,0,0,0,0,0,0,0,0,0,0,0
Closing Balance,0,0,0,0,0,0,0,0,0,0,0,0


In [418]:
balance_sheet_df.loc['Inventories']  = inventories_schedule.loc['Closing Balance']

### **Equity and Intercompany Loans**


In [419]:
balance_sheet_df.loc["Issued Share Capital"] = helper.change_period_index_to_strftime(
    parameters.loc["SHARE_CAPITAL"].cumsum()
    + opening_balances["ISSUED_SHARE_CAPITAL"].iat[0]
)

balance_sheet_df.loc["Intercompany Loans"] = helper.change_period_index_to_strftime(
    parameters.loc["INTERCOMPANY_LOANS"].cumsum()
    + opening_balances["INTERCOMPANY_LOANS"].iat[0]
)


balance_sheet_df.loc["Share Premium"] = helper.change_period_index_to_strftime(
    parameters.loc["SHARE_PREMIUM"].cumsum() + opening_balances["SHARE_PREMIUM"].iat[0]
)

balance_sheet_df.loc[
    "Other Components Of Equity"
] = helper.change_period_index_to_strftime(
    parameters.loc["OTHER_COMPONENTS_OF_EQUITY"].cumsum()
    + opening_balances["OTHER_COMPONENTS_OF_EQUITY"].iat[0]
)

balance_sheet_df.loc["Treasury Shares"] = helper.change_period_index_to_strftime(
    parameters.loc["TREASURY_SHARES"].cumsum()
    + opening_balances["TREASURY_SHARES"].iat[0]
)

balance_sheet_df.loc["Retained Earnings"] = (
    income_statement_df.loc["PROFIT/(LOSS) FOR PERIOD"]
    - helper.change_period_index_to_strftime(parameters.loc["DIVIDEND_PAID"])
).cumsum()

balance_sheet_df.loc["Capital And Reserves"] = balance_sheet_df.loc[
    "Issued Share Capital":"Retained Earnings"
].sum()

### **Other Assets**

In [420]:
balance_sheet_df.loc["Intangible Assets"] = helper.change_period_index_to_strftime(
    parameters.loc["INTANGIBLE_ASSETS"].cumsum()
    + opening_balances["INTANGIBLE_ASSETS"].iat[0]
)

balance_sheet_df.loc[
    "Investment In Subsidiaries"
] = helper.change_period_index_to_strftime(
    parameters.loc["INVESTMENT_IN_SUBSIDIARIES"].cumsum()
    + opening_balances["INVESTMENT_IN_SUBSIDIARIES"].iat[0]
)

balance_sheet_df.loc[
    "Investment In Associates"
] = helper.change_period_index_to_strftime(
    parameters.loc["INVESTMENT_IN_ASSOCIATES"].cumsum()
    + opening_balances["INVESTMENT_IN_ASSOCIATES"].iat[0]
)

balance_sheet_df.loc["Investment Properties"] = helper.change_period_index_to_strftime(
    parameters.loc["INVESTMENT_PROPERTIES"].cumsum()
    + opening_balances["INVESTMENT_PROPERTIES"].iat[0]
)

balance_sheet_df.loc["Equity Investments"] = helper.change_period_index_to_strftime(
    parameters.loc["EQUITY_INVESTMENTS"].cumsum()
    + opening_balances["EQUITY_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Long Term Money Market Investments"
] = helper.change_period_index_to_strftime(
    parameters.loc["LONG_TERM_MONEY_MARKET_INVESTMENTS"].cumsum()
    + opening_balances["LONG_TERM_MONEY_MARKET_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Short Term Money Market Investments"
] = helper.change_period_index_to_strftime(
    parameters.loc["SHORT_TERM_MONEY_MARKET_INVESTMENTS"].cumsum()
    + opening_balances["SHORT_TERM_MONEY_MARKET_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Loans To Related Entities"
] = helper.change_period_index_to_strftime(
    parameters.loc["LOANS_TO_RELATED_ENTITIES"].cumsum()
    + opening_balances["LOANS_TO_RELATED_ENTITIES"].iat[0]
)

In [421]:
balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"147,742,347.95","144,762,481.73","141,782,615.51","138,802,749.29","135,822,883.07","132,843,016.84","129,863,150.62","126,883,284.40","123,903,418.18","120,923,551.96","117,947,239.27","114,970,610.52"
Intangible Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Investment In Subsidiaries,0.00,"-1,000.00","-1,000.00","-1,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00"
Investment In Associates,"10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","11,000.00","11,000.00","11,000.00","11,000.00","11,000.00","11,000.00"
Investment Properties,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,000.00","5,000.00","5,000.00","5,000.00"
Equity Investments,"20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","21,000.00","21,000.00","21,000.00","21,000.00","21,000.00","21,000.00","31,000.00"
Long Term Money Market Investments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### **Calculate Totals**


In [422]:
balance_sheet_df = balance_sheet.sum_financial_statements_totals(balance_sheet_df)
balance_sheet_df = balance_sheet.calculate_final_balances(balance_sheet_df=balance_sheet_df)
balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"147,742,347.95","144,762,481.73","141,782,615.51","138,802,749.29","135,822,883.07","132,843,016.84","129,863,150.62","126,883,284.40","123,903,418.18","120,923,551.96","117,947,239.27","114,970,610.52"
Intangible Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Investment In Subsidiaries,0.00,"-1,000.00","-1,000.00","-1,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00","9,000.00"
Investment In Associates,"10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","11,000.00","11,000.00","11,000.00","11,000.00","11,000.00","11,000.00"
Investment Properties,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,000.00","5,000.00","5,000.00","5,000.00"
Equity Investments,"20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","21,000.00","21,000.00","21,000.00","21,000.00","21,000.00","21,000.00","31,000.00"
Long Term Money Market Investments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## **Indirect Cashflow**

In [423]:
statement_of_cashflow_df = (
    statement_of_cashflows.generate_statement_of_cashflow_template(
        VALUATION_DATE, MONTHS_TO_FORECAST
    )
)

### **Insert Available Balances**

In [424]:
statement_of_cashflow_df.loc["Profit/(loss) Per I/S"] = income_statement_df.loc[
    "PROFIT/(LOSS) FOR PERIOD"
]
statement_of_cashflow_df.loc["Depreciation"] = income_statement_df.loc["Depreciation"]
statement_of_cashflow_df.loc["Dividend Paid"] = -helper.change_period_index_to_strftime(
    parameters.loc["DIVIDEND_PAID"]
)
statement_of_cashflow_df.loc[
    "Treasury Movements"
] = helper.change_period_index_to_strftime(parameters.loc["TREASURY_MOVEMENTS"])
statement_of_cashflow_df.loc[
    "Interest Expense Accrued"
] = helper.change_period_index_to_strftime(parameters.loc["INTEREST_EXPENSE_ACCRUED"])
statement_of_cashflow_df.loc[
    "Other Non-Cash Items"
] = helper.change_period_index_to_strftime(parameters.loc["OTHER_NON_CASH_ITEMS"])
statement_of_cashflow_df.loc["Interest Paid"] = -finance_costs_df.loc["total"]
statement_of_cashflow_df.loc["Tax Paid"] = tax_schedule.loc["Tax Paid"]
statement_of_cashflow_df.loc[
    "Repayment Of Borrowings"
] = -capital_repayment_borrowings_df.loc["total"]
statement_of_cashflow_df.loc["Purchase Of Fixed Assets"] = capital_expenses
statement_of_cashflow_df.loc["Increase/(Decrease) In Short Term Borrowings"] = (
    short_term_loans_schedules.loc["Closing Balance"]
    - short_term_loans_schedules.loc["Opening Balance"]
)
statement_of_cashflow_df.loc["Increase/(Decrease) In Long Term Borrowings"] = (
    long_term_loans_schedules.loc["Closing Balance"]
    - long_term_loans_schedules.loc["Opening Balance"]
)
statement_of_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_CASHFLOWS,,,,,,,,,,,,
Profit/(loss) Per I/S,"-3,268,442.82","-3,213,347.89","-3,217,672.87","-3,125,899.80","-3,078,732.99","-3,120,540.81","-3,042,020.16","-3,028,500.41","-3,048,075.67","-2,933,765.37","-2,859,366.62","-2,798,127.39"
Treasury Movements,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Adjustments For:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Interest Expense Accrued,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other Non-Cash Items,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cash From Operations Before WC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Working Capital Movements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Increase)/Decrease In Receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [425]:
statement_of_cashflow_df.loc[
    "Cash From Operations Before WC"
] = statement_of_cashflow_df.iloc[
    1 : statement_of_cashflow_df.index.get_loc("Cash From Operations Before WC")
].sum()

In [426]:

statement_of_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_CASHFLOWS,,,,,,,,,,,,
Profit/(loss) Per I/S,"-3,268,442.82","-3,213,347.89","-3,217,672.87","-3,125,899.80","-3,078,732.99","-3,120,540.81","-3,042,020.16","-3,028,500.41","-3,048,075.67","-2,933,765.37","-2,859,366.62","-2,798,127.39"
Treasury Movements,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Adjustments For:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Interest Expense Accrued,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other Non-Cash Items,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cash From Operations Before WC,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Working Capital Movements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Increase)/Decrease In Receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Working Capital Movements**

In [427]:
change_in_receivables = (
    other_receivables_schedule.loc["Closing Balance"]
    - other_receivables_schedule.loc["Opening Balance"]
    + trade_receivables_schedule.loc["Closing Balance"]
    - trade_receivables_schedule.loc["Opening Balance"]
    + intergroup_receivables_schedule.loc["Closing Balance"]
    - intergroup_receivables_schedule.loc["Opening Balance"]
)
change_in_receivables

Jan-2023   -500
Feb-2023   -500
Mar-2023   -500
Apr-2023   -500
May-2023   -500
Jun-2023   -500
Jul-2023   -500
Aug-2023   -500
Sep-2023   -500
Oct-2023   -500
Nov-2023   -500
Dec-2023   -500
dtype: int64

In [428]:
change_in_payables = (
    trade_payables_schedule.loc["Closing Balance"]
    - trade_payables_schedule.loc["Opening Balance"]
    + other_payables_schedule.loc["Closing Balance"]
    - other_payables_schedule.loc["Opening Balance"]
)
change_in_payables

Jan-2023   -500
Feb-2023   -500
Mar-2023   -500
Apr-2023   -500
May-2023   -500
Jun-2023   -500
Jul-2023   -500
Aug-2023   -500
Sep-2023   -500
Oct-2023   -500
Nov-2023   -500
Dec-2023   -500
dtype: int64

In [429]:
change_in_loan_book_principle = loan_book_df.loc["New Disbursements":"Repayments"].sum()
change_in_loan_book_principle

Jan-2023   337,498.41
Feb-2023   344,515.75
Mar-2023   319,318.36
Apr-2023    76,012.37
May-2023    39,032.04
Jun-2023    35,497.38
Jul-2023    44,062.54
Aug-2023   193,197.41
Sep-2023   305,916.66
Oct-2023   501,259.29
Nov-2023   466,708.32
Dec-2023   264,399.96
dtype: object

In [430]:
change_in_loan_book_interest = loan_book_df.loc['Interest Income']
change_in_loan_book_interest

Jan-2023     3,925.05
Feb-2023    48,488.03
Mar-2023    97,752.56
Apr-2023   147,475.18
May-2023   179,319.33
Jun-2023   203,927.28
Jul-2023   227,565.56
Aug-2023   250,100.30
Sep-2023   287,972.79
Oct-2023   334,165.97
Nov-2023   403,169.47
Dec-2023   475,340.40
Name: Interest Income, dtype: object

In [431]:
borrowings_schedule = long_term_loans_schedules + short_term_loans_schedules
change_in_borrowings = borrowings_schedule.loc['Closing Balance'] - borrowings_schedule.loc['Opening Balance']
change_in_borrowings

Jan-2023    -3,373.20
Feb-2023    -3,346.42
Mar-2023   -54,518.45
Apr-2023    -3,752.40
May-2023    -3,635.24
Jun-2023   -59,949.94
Jul-2023    -3,397.55
Aug-2023    -3,276.99
Sep-2023   -54,076.56
Oct-2023    -1,610.77
Nov-2023    -1,582.79
Dec-2023   706,090.60
dtype: object

In [432]:
statement_of_cashflow_df.loc['(Increase)/Decrease In Receivables'] = change_in_receivables
statement_of_cashflow_df.loc['Increase/(Decrease) In Payables'] = change_in_payables
statement_of_cashflow_df.loc['(Increase)/Decrease In Loan Book (Principle)'] = change_in_loan_book_principle
statement_of_cashflow_df.loc['(Increase)/Decrease In Loan Book (Interest)'] = change_in_loan_book_interest
statement_of_cashflow_df.loc['Increase/(Decrease) In Borrowings'] = change_in_borrowings

In [433]:
statement_of_cashflow_df.loc["Cash From Operations After WC"] = (
    statement_of_cashflow_df.loc["Cash From Operations Before WC"]
    + statement_of_cashflow_df.iloc[
        statement_of_cashflow_df.index.get_loc("Working Capital Movements")
        + 1 : statement_of_cashflow_df.index.get_loc("Cash From Operations After WC")
    ].sum()
)

In [434]:
statement_of_cashflow_df.loc[
    "Net Cash Flow From Operations"
] = statement_of_cashflow_df.loc["Cash From Operations After WC":"Tax Paid"].sum()

In [435]:
statement_of_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_CASHFLOWS,,,,,,,,,,,,
Profit/(loss) Per I/S,"-3,268,442.82","-3,213,347.89","-3,217,672.87","-3,125,899.80","-3,078,732.99","-3,120,540.81","-3,042,020.16","-3,028,500.41","-3,048,075.67","-2,933,765.37","-2,859,366.62","-2,798,127.39"
Treasury Movements,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Adjustments For:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Interest Expense Accrued,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other Non-Cash Items,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cash From Operations Before WC,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Working Capital Movements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Increase)/Decrease In Receivables,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500


### **Calculating Totals**

In [436]:
statement_of_cashflow_df = balance_sheet.sum_financial_statements_totals(statement_of_cashflow_df)


statement_of_cashflow_df.loc['Net Increase/(Decrease) In Cash'] = (
    statement_of_cashflow_df.loc["Net Cash Flow From Operations"]
    + statement_of_cashflow_df.loc["Cash Flow From Investing Activities"]
    + statement_of_cashflow_df.loc["Cash Flow From Financing Activities"]
)
statement_of_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_CASHFLOWS,,,,,,,,,,,,
Profit/(loss) Per I/S,"-3,268,442.82","-3,213,347.89","-3,217,672.87","-3,125,899.80","-3,078,732.99","-3,120,540.81","-3,042,020.16","-3,028,500.41","-3,048,075.67","-2,933,765.37","-2,859,366.62","-2,798,127.39"
Treasury Movements,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Adjustments For:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Interest Expense Accrued,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other Non-Cash Items,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cash From Operations Before WC,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Working Capital Movements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Increase)/Decrease In Receivables,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500


### **Calculating Cash at End and Beginning of Period**

In [437]:
statement_of_cashflow_df = (
    statement_of_cashflows.calculate_cash_at_end_and_beginning_of_period(
        statement_of_cashflow_df=statement_of_cashflow_df,
        opening_balances=opening_balances,
    )
)

statement_of_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_CASHFLOWS,,,,,,,,,,,,
Profit/(loss) Per I/S,"-3,268,442.82","-3,213,347.89","-3,217,672.87","-3,125,899.80","-3,078,732.99","-3,120,540.81","-3,042,020.16","-3,028,500.41","-3,048,075.67","-2,933,765.37","-2,859,366.62","-2,798,127.39"
Treasury Movements,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Adjustments For:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Interest Expense Accrued,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other Non-Cash Items,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cash From Operations Before WC,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Working Capital Movements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Increase)/Decrease In Receivables,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500,-500
